# Установка библиотек и импорт их


In [ ]:
#!pip install ccxt
!pip install python-binance
!pip install binance
!pip install -U kaleido
! pip install ipywidgets

In [ ]:
!pip install vectorbt

In [1]:
import pandas as pd
import datetime

#from numba import jit
import numpy as np
import seaborn as sns
import time
#import datetime
#import  ccxt 
import requests
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager#
from futures_sign import send_signed_request, send_public_request
from cred import KEY, SECRET


import chart_studio
import chart_studio.plotly as ply
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot, init_notebook_mode

from __future__ import print_function

import pybacktest

#from google.colab import drive
#drive.mount('/content/drive')

import warnings


from sklearn.cluster import MeanShift,SpectralClustering,Birch,OPTICS
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, AffinityPropagation
from sklearn.metrics import adjusted_rand_score, silhouette_score
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram
    

# Все функции


In [2]:
# To find a slope of price line 
def indSlope(series,n):
    
    array_sl = [j*0 for j in range(n-1)]
    
    for j in range(n,len(series)+1):
        y = series[j-n:j]
        x = np.array(range(n))
        x_sc = (x - x.min())/(x.max() - x.min())
        y_sc = (y - y.min())/(y.max() - y.min())
        x_sc = sm.add_constant(x_sc)
        model = sm.OLS(y_sc,x_sc)
        results = model.fit()
        array_sl.append(results.params[-1])
    slope_angle = (np.rad2deg(np.arctan(np.array(array_sl))))
    return np.array(slope_angle)

# True Range and Average True Range indicator

def indATR(source_DF,n):
    df = source_DF.copy()
    df['H-L']=abs(df['high']-df['low'])
    df['H-PC']=abs(df['high']-df['close'].shift(1))
    df['L-PC']=abs(df['low']-df['close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    df_temp = df.drop(['H-L','H-PC','L-PC'],axis=1)
    return df_temp

# generate data frame with all needed data

def PrepareDF(DF):
    ohlc = DF.iloc[:,[0,1,2,3,4,5]]
    ohlc.columns = ["date","open","high","low","close","volume"]
    ohlc=ohlc.set_index('date')
    df = indATR(ohlc,14).reset_index()
    df['slope'] = indSlope(df['close'],5)
    df['channel_max'] = df['high'].rolling(10).max()
    df['channel_min'] = df['low'].rolling(10).min()
    df['position_in_channel'] = (df['close']-df['channel_min']) / (df['channel_max']-df['channel_min'])
    df = df.set_index('date')
    df = df.reset_index()
    return(df)
# find local mimimum / local maximum

def isLCC(DF,i):
    df=DF.copy()
    LCC=0
    
    if df['close'][i]<=df['close'][i+1] and df['close'][i]<=df['close'][i-1] and df['close'][i+1]>df['close'][i-1]:
        #найдено Дно
        LCC = i-1;
    return LCC

def isHCC(DF,i):
    df=DF.copy()
    HCC=0
    if df['close'][i]>=df['close'][i+1] and df['close'][i]>=df['close'][i-1] and df['close'][i+1]<df['close'][i-1]:
        #найдена вершина
        HCC = i;
    return HCC


def getMaxMinChannel(DF, n):
    maxx=0
    minn=DF['low'].max()
    for i in range (1,n):
        if maxx<DF['high'][len(DF)-i]:
            maxx=DF['high'][len(DF)-i]
        if minn>DF['low'][len(DF)-i]:
            minn=DF['low'][len(DF)-i]
    return(maxx,minn)

In [5]:
#@title  { form-width: "1px", display-mode: "code" }
def chartTel(data,sym='',napr=''):
    #declare figure

    #graph = go.Figure()
    #Adding Trace
    graph = make_subplots(specs=[[{"secondary_y": True}]])
    graph.add_trace(go.Candlestick(x=data.open_time,
                       open=data.open,
                       high=data.high,
                       low=data.low,
                       close=data.close, name = sym), secondary_y=True)
    graph.add_trace(go.Scatter(x=data.open_time, y=data['DeltHLMA'],name='DeltHLMA'), secondary_y=False)
    graph.add_trace(go.Scatter(x=data.open_time, y=data['DeltHL'],name='DeltHL'), secondary_y=False)
    graph.add_trace(go.Scatter(x=data.open_time, y=data['DeltCO'].rolling(5).mean(),name='DeltCO'), secondary_y=False)
    graph.add_trace(go.Scatter(x=data.ope|n_time, y=data['DeltHO'],name='DeltHO'), secondary_y=False)
    graph.add_trace(go.Scatter(x=data.open_time, y=data['DeltOL'],name='DeltOL'), secondary_y=False)
    graph.add_trace(go.Scatter(x=data.open_time, y=(data['DeltCL']),name='DeltCL'), secondary_y=False)
    graph.add_trace(go.Scatter(x=data.open_time, y=(data['DeltHC']),name='DeltHC'), secondary_y=False)

    graph.add_hline(0)
    graph.layout.yaxis2.showgrid=False

    graph.write_image(Im+sym+".png") 
    
def sendImage(sym):
    #отправляем картинку
    bot_token2 = bot_token
    bot_chatID = chat_id
    url = "https://api.telegram.org/bot"+ bot_token2 +"/sendPhoto";
     
    files = {'photo': open(Im+sym+".png", 'rb')}
    data = {'chat_id' : bot_chatID}
    r= requests.post(url, files=files, data=data)
    #print(r.status_code, r.reason, r.content)
    #отправляем картинку

#   ВСЕ ФУНКЦИИ
def get_symbol_price(symbol):
    try:
        prices = client.get_all_tickers()
    except:
        time.sleep(2)
        print('Исключение3')
        try:
            prices = client.get_all_tickers()
        except:
            time.sleep(5)
            prices = client.get_all_tickers()
    df8=pd.DataFrame(prices)
    #df8.price=pd.to_numeric(df8.price)
    df8.price=df8['price'].apply(lambda x: float(x))
    sym=str(symbol)
    mm=df8.loc[df8['symbol'] ==sym , 'price'].values[0]
    return mm



def VBpoisk(candles):
    verh=0
    niz =999999999
    for i in candles.index:
        # Внешний бар ищем 
        if i <=df.index[-4]: 
            if candles.high[i]>=candles.close[i+2] and candles.low[i]<=candles.close[i+2] and candles.high[i]>=candles.close[i+1] and candles.low[i]<=candles.close[i+1]:
                verh=candles.high[i]
                niz =candles.low[i]
        if i <df.index[-1]:
            if (candles.close[i]>verh and candles.close[i+1]>verh and verh>0)or (candles.close[i]<niz and candles.close[i+1]<niz and niz<999999999):
                verh=0
                niz =999999999            
        if i ==df.index[-1]  and candles.close[i]<=verh and candles.close[i]>=niz:
            #print('False'+str(verh)+str(niz)+str(candles.close[i]))
            return False
        elif (i ==df.index[-1] or i ==df.index[-2]) and( candles.close[i]>verh or candles.close[i]<niz): 
            #print('True'+str(verh)+str(niz)+str(candles.close[i]))
            return True
 

  # Open position for Sybol with 
def open_position(symbol,s_l,quantity_l):
    sprice=get_symbol_price(symbol)

    if(s_l=='long'):
        close_price=str(round(sprice*(1+0.01),get_count(sprice)))
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "BUY",
                    "type": "LIMIT",
                    "quantity": str(quantity_l),
                    "timeInForce":"GTC",
                    "price": close_price        

                }
            ]
        }
        responce = send_signed_request('POST', '/fapi/v1/batchOrders', params)
       
    if(s_l=='short'):
        close_price=str(round(sprice*(1-0.01),get_count(sprice)))
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "SELL",
                    "type": "LIMIT",
                    "quantity": str(quantity_l),
                    "timeInForce":"GTC",
                    "price": close_price
                }
           ]
        }
        responce = send_signed_request('POST', '/fapi/v1/batchOrders', params)
# Close position for symbol with quantity

def close_position(symbol,s_l,quantity_l):
    sprice=get_symbol_price(symbol)

    if(s_l=='long'):
        close_price=str(round(sprice*(1-0.01),get_count(sprice)))
        params = {
                    "symbol":symbol,
                    "side": "SELL",
                    "type": "LIMIT",
                    "quantity": str(quantity_l),
                    "timeInForce":"GTC",
                    "price": close_price
                }
        responce = send_signed_request('POST', '/fapi/v1/order', params)
    if(s_l=='short'):
        close_price=str(round(sprice*(1+0.01),get_count(sprice)))
        params = {
                
                    "symbol":symbol,
                    "side": "BUY",
                    "type": "LIMIT",
                    "quantity": str(quantity_l),
                    "timeInForce":"GTC",
                    "price": close_price        
                }
        responce = send_signed_request('POST', '/fapi/v1/order', params)

def get_count(number):
    s = str(number)
    if '.' in s:
        return abs(s.find('.') - len(s)) - 1
    else:
        return 0
# Close all orders 

def check_and_close_orders(symbol):
    global isStop 
    a=client.futures_get_open_orders(symbol=symbol)
    if len(a)>0:
        isStop = False
        client.futures_cancel_all_open_orders(symbol=symbol)

# Find all opened positions

def get_opened_positions(symbol):
    status = client.futures_account()
    positions=pd.DataFrame(status['positions'])
    a = positions[positions['symbol']==symbol]['positionAmt'].astype(float).tolist()[0]
    leverage = int(positions[positions['symbol']==symbol]['leverage'])
    entryprice = positions[positions['symbol']==symbol]['entryPrice']
    profit = float(status['totalUnrealizedProfit'])
    balance = round(float(status['totalWalletBalance']),2)
    if a>0:
        pos = "long"
    elif a<0:
        pos = "short"
    else: 
        pos = ""
    return([pos,a,profit,leverage,balance,round(float(entryprice),3),0])

 # Get last 500 kandels 5 minutes for Symbol
# Get

def get_spot_klines(symbol,timefr,limit=4):
    x = requests.get('https://binance.com/api/v3/klines?symbol='+symbol+'&limit='+str(limit)+'&interval='+timefr)
    df=pd.DataFrame(x.json())
    
    df.columns=['open_time','O','H','L','C','V','close_time','d1','d2','d3','d4','d5']
    df=df.drop(['d1','d2','d3','d4','d5'],axis=1)
    df['O']=df['O'].astype(float)
    df['H']=df['H'].astype(float)
    df['L']=df['L'].astype(float)
    df['C']=df['C'].astype(float)
    df['V']=df['V'].astype(float)
    df['open_time']=df['open_time'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000.0))
    
    df['close_time']=df['close_time'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000.0))
    df.open_time= pd.DatetimeIndex(df['open_time'])
    df=df. set_index ('open_time')
    return(df)

def get_futures_klines(symbol,timefr,limit=4):
    x = requests.get('https://binance.com/fapi/v1/klines?symbol='+symbol+'&limit='+str(limit)+'&interval='+timefr)
    df=pd.DataFrame(x.json())
    
    df.columns=['open_time','O','H','L','C','V','close_time','d1','d2','d3','d4','d5']
    df=df.drop(['d1','d2','d3','d4','d5'],axis=1)
    df['O']=df['O'].astype(float)
    df['H']=df['H'].astype(float)
    df['L']=df['L'].astype(float)
    df['C']=df['C'].astype(float)
    df['V']=df['V'].astype(float)
    df['open_time']=df['open_time'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000.0))
    
    df['close_time']=df['close_time'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000.0))
    df.open_time= pd.DatetimeIndex(df['open_time'])
    df=df. set_index ('open_time')
    
    #df=df(index='open_time')

    return(df)

def podgData1(df):
    
    #df1=pd.read_csv('/content/drive/MyDrive/history/'+symb+timefr+'Obrabotka.csv')
    #df2 =get_futures_klines(symb,timefr,10) 

    
    
    ma=max(df['H'])
    vo=max(df['volume'])
    op=df.open[1]
    df.open=df.open/ma
    df.close=df.close/ma
    df.high=df.high/ma
    df.low=df.low/ma
    op1=df.close.mean()
    df.open=df.open-op1
    df.close=df.close-op1
    df.high=df.high-op1
    df.low=df.low-op1
    df.volume=df.volume/vo
    df=pd.concat([df, pd.DataFrame(columns = ['mediana','DeltHL','DeltCO','DeltHC','DeltHO','DeltCL','DeltOL','DeltHLMA',
                                              'DeltHLCO',
                                              #'slopeHL','slopeHO','slopeHC','slopeCO','slopeCL','slopeOL',
                                              'SummPrice','VolumMEAN','volumMA','ClMed','MedOp','SummMedian',
                                              'DeltCL_HC','DeltHO_OL','SummDelt'])])
    df['mediana'] =(df.high+df.low)/2
    df['DeltHL'] =(df.high-df.low).rolling(3).mean()
    df['DeltHLMA'] =(df.high-df.low).mean()
    df['DeltCO'] =(df.close-df.open).rolling(3).mean()
    df['DeltHC'] =(df.high-df.close).rolling(3).mean()
    df['DeltHO'] =(df.high-df.open).rolling(3).mean()
    df['DeltCL'] =(df.close-df.low).rolling(3).mean()
    df['DeltOL'] =(df.open-df.low).rolling(3).mean()
    df.fillna(0)
    
    df['Long'] =df['DeltHO']>df['DeltCL']>df['DeltHC']>df['DeltOL']
    df['Short'] =df['DeltHO']<df['DeltCL']<df['DeltHC']<df['DeltOL']
    
    #df['OtnHLCO'] =((df['DeltCO'])/(df['DeltHL']))*((df['DeltCO'])/(df['DeltHL']))
 
      

    df['VolumMEAN'] =(df.volume.mean())
    df['volumMA']= df.volume.rolling(5).mean()
    df['ClMed'] =(df['close']-df['mediana']) #вверх или вниз
    df['MedOp'] =(df['mediana']-df['open']) #вверх или вниз
    df['SummMedian']=df['ClMed']+df['MedOp']
    df['DeltHC_OL']= (df['DeltHC']-df['DeltOL']) #вверх или вниз
    df['DeltHO_CL']=(df['DeltHO']-df['DeltCL']) #вверх или вниз
    #df['SummDelt']=df['DeltCL_HC']+df['DeltHO_OL']
    df.fillna(0)
    
    
    
    return(df)  

def SignalVh(df):
    if (df.DeltHL[(df.index[-1])]<df.DeltHLMA[(df.index[-1])]  )and df.DeltHL[(df.index[-1])]>df.DeltHL[(df.index[-2])]:
      sig='    Заходим '
      p=1
    else:
      sig='    Ждем '
      p=0
    #print(sig)  
    return  p 

def Signal11(df,sy,i): # new 27.07.22

    s=0
    if df.DeltCO[(df.index[-3])]<df.DeltCO[(df.index[-2])]:  
            ord.at[sy,'SignalTime']=df.open_time[(df.index[i])]  
            ord.at[sy,'Signal']=1
            s=1
            return (s)
    elif  df.DeltCO[(df.index[-3])]>df.DeltCO[(df.index[-2])]:
            s=-1
            ord.at[sy,'Signal']=-1
            ord.at[sy,'SignalTime']=df.open_time[(df.index[i])]  
            return (s)
    else:
            s=0
            ord.at[sy,'Signal']=0
            ord.at[sy,'SignalTime']=df.open_time[(df.index[i])]        
            return (s)
    ord.at[sy,'Signal']=0
    ord.at[sy,'SignalTime']=df.open_time[(df.index[i])] 
        
    return (s)


def Signal00(df,sy,i):

    s=0
    #if df.DeltCL[(df.index[i-1])]<df.DeltCL[(df.index[i])] and df.DeltOL[(df.index[i])]<df.DeltCL[(df.index[i])]:  
    if df.DeltHC[(df.index[i])]<df.DeltCL[(df.index[i])] and df.DeltCO[(df.index[i])]>-0.01 and \
    df.DeltOL[(df.index[i])]<df.DeltHO[(df.index[i])] and df.DeltOL[(df.index[i])]<df.DeltCL[(df.index[i])]\
    and df.DeltHC[(df.index[i])]<df.DeltHO[(df.index[i])]and df.DeltHC[(df.index[i])]<df.DeltOL[(df.index[i])]:#\

            ord.at[sy,'SignalTime']=df.index[i]  
            ord.at[sy,'Signal']=1
            s=1
            #print(symb+str(tks.at[sy,'deltaVol'])+ ' = DeltaVol   Signal= '+ str(ord.at[sy,'Signal'])+' Order= '+str(ord.at[sy,'Order']))
            #print(' ***покупаем на все ')
            #print(HLMA,HL2,HL1,HLCOMA,HLCO2,HLCO1)
            return (s)
    #elif  df.DeltHC[(df.index[i])]>df.DeltHO[(df.index[i])] and df.DeltHC[(df.index[i])]>df.DeltHC[(df.index[i-1])]:
    elif  df.DeltHC[(df.index[i])]>df.DeltCL[(df.index[i])] and df.DeltCO[(df.index[i])]<0.01 \
    and df.DeltOL[(df.index[i])]>df.DeltHO[(df.index[i])] and df.DeltOL[(df.index[i])]>df.DeltCL[(df.index[i])]\
    and df.DeltHC[(df.index[i])]>df.DeltHO[(df.index[i])] and df.DeltHC[(df.index[i])]>df.DeltOL[(df.index[i])]:

            #tks.at[sy,timefr]=-2
            s=-1
            ord.at[sy,'Signal']=-1
            ord.at[sy,'SignalTime']=df.index[i]  
            #print('    продаем на все*** ')
            #print(HLMA,HL2,HL1,HLCOMA,HLCO2,HLCO1)
            return (s)
    else:
    #elif  df.DeltHC[(df.index[i])]>df.DeltCL[(df.index[i])] and df.DeltHO[(df.index[i])]>df.DeltOL[(df.index[i])] \
    #or df.DeltHC[(df.index[i])]<df.DeltCL[(df.index[i])] and df.DeltHO[(df.index[i])]<df.DeltOL[(df.index[i])]:
            #tks.at[sy,timefr]=0
            s=0
            ord.at[sy,'Signal']=0
            ord.at[sy,'SignalTime']=df.index[i]       
            #print(' Шляпа какая-то ')
            #print(HLMA,HL2,HL1,HLCOMA,HLCO2,HLCO1)
            return (s)
    ord.at[sy,'Signal']=0
    ord.at[sy,'SignalTime']=df.index[i] 
    #print(' Шляпа какая-то ')        
    return (s)



def Signal101(df,sy):
  
    if df.DeltHC[(df.index[-1])]<df.DeltCL[(df.index[-1])] and df.DeltCO[(df.index[-1])]>-0.01 and \
    df.DeltOL[(df.index[-1])]<df.DeltHO[(df.index[-1])] and df.DeltOL[(df.index[-1])]<df.DeltCL[(df.index[-1])]\
    and df.DeltHC[(df.index[-1])]<df.DeltHO[(df.index[-1])]:#\

            tks.at[sy,timefr]=2
            ord.at[sy,'SignalTime']=str(time.ctime())
            ord.at[sy,'Signal']=2
            #print(symb+str(tks.at[sy,'deltaVol'])+ ' = DeltaVol   Signal= '+ str(ord.at[sy,'Signal'])+' Order= '+str(ord.at[sy,'Order']))
            print(' ***покупаем на все***')
            return True

    elif  df.DeltHC[(df.index[-1])]>df.DeltCL[(df.index[-1])] and df.DeltCO[(df.index[-1])]<0.01 \
    and df.DeltOL[(df.index[-1])]>df.DeltHO[(df.index[-1])] and df.DeltOL[(df.index[-1])]>df.DeltCL[(df.index[-1])]\
    and df.DeltHC[(df.index[-1])]>df.DeltHO[(df.index[-1])]:
            tks.at[sy,timefr]=-2
            
            ord.at[sy,'Signal']=-2
            ord.at[sy,'SignalTime']=str(time.ctime())
            print('                               ***продаем на все***')
            return True

    else:  
            tks.at[sy,timefr]=0
            
            ord.at[sy,'Signal']=0
            ord.at[sy,'SignalTime']=str(time.ctime())        
            print('                                                          Шляпа какая-то')
    return True



def Signal1(df):
    #if df.volume[(df.index[-1])]>=df.VolumMEAN[(df.index[-1])] or df.volume[(df.index[-2])]>=df.VolumMEAN[(df.index[-2])]:
    #if df.SummDelt[(df.index[-2])]<df.SummDeltMEAN[(df.index[-2])]or df.volumMA[(df.index[-1])]>df.VolumMEAN[(df.index[-2])]or df.volumMA[(df.index[-3])]>df.VolumMEAN[(df.index[-2])]: 
        
    if df.DeltHO[(df.index[-1])]>df.DeltCL[(df.index[-1])]> df.DeltHC[(df.index[-1])]>df.DeltOL[(df.index[-1])]\
     and df.DeltCL[(df.index[-2])]<df.DeltCL[(df.index[-1])] and df.DeltHO[(df.index[-1])]>df.DeltHO[(df.index[-2])] \
    and df.DeltHC[(df.index[-2])]>df.DeltHC[(df.index[-1])] and df.DeltOL[(df.index[-1])]<df.DeltOL[(df.index[-2])]:
     #and df.slopeHO[(df.index[-1])]>df.slopeCL[(df.index[-1])]>0 and 0>df.slopeHC[(df.index[-1])]> df.slopeOL[(df.index[-1])] :# and indSlope(df.DeltHO,4)>indSlope(df.DeltCL,4)>indSlope(df.DeltHC,4)>indSlope(df.DeltOL,4)  :
            #zapis(symb,tf,1)
            tks.at[sy,timefr]=2
            ord.at[sy,'SignalTime']=str(time.ctime())
            ord.at[sy,'Signal']=2
            #print(symb+str(tks.at[sy,'deltaVol'])+ ' = DeltaVol   Signal= '+ str(ord.at[sy,'Signal'])+' Order= '+str(ord.at[sy,'Order']))
            print(' ***покупаем на все***')
            return True
    elif  (df.DeltCL[(df.index[-1])]>df.DeltHC[(df.index[-1])]and df.DeltHO[(df.index[-1])]>df.DeltOL[(df.index[-1])]and \
           df.DeltHO[(df.index[-1])]>df.DeltHC[(df.index[-1])]and df.DeltCL[(df.index[-1])]>df.DeltOL[(df.index[-1])]):
            #zapis(symb,tf,1)
            tks.at[sy,timefr]=1
            ord.at[sy,'SignalTime']=str(time.ctime())
            ord.at[sy,'Signal']=1
            #print(symb+str(tks.at[sy,'deltaVol'])+ ' = DeltaVol   Signal= '+ str(ord.at[sy,'Signal'])+' Order= '+str(ord.at[sy,'Order']))
            #print('     держим покупки')
            return True
    elif  df.DeltHO[(df.index[-1])]<df.DeltCL[(df.index[-1])]< df.DeltHC[(df.index[-1])]<df.DeltOL[(df.index[-1])] \
     and df.DeltCL[(df.index[-2])]>df.DeltCL[(df.index[-1])] and df.DeltHO[(df.index[-1])]<df.DeltHO[(df.index[-2])]  and df.DeltHC[(df.index[-2])]<df.DeltHC[(df.index[-1])] and df.DeltOL[(df.index[-1])]>df.DeltOL[(df.index[-2])]:
     #and df.slopeHO[(df.index[-1])]<df.slopeCL[(df.index[-1])]<0 and 0<df.slopeHC[(df.index[-1])]<df.slopeOL[(df.index[-1])] :#and indSlope(df.DeltHO,4)<indSlope(df.DeltCL,4)<indSlope(df.DeltHC,4)<indSlope(df.DeltOL,4) :
            tks.at[sy,timefr]=-2
            
            ord.at[sy,'Signal']=-2
            ord.at[sy,'SignalTime']=str(time.ctime())
            print('                               ***продаем на все***')
            return True
    elif   (df.DeltCL[(df.index[-1])]<df.DeltHC[(df.index[-1])]and df.DeltHO[(df.index[-1])]<df.DeltOL[(df.index[-1])]and \
            df.DeltHO[(df.index[-1])]<df.DeltHC[(df.index[-1])]and df.DeltCL[(df.index[-1])]<df.DeltOL[(df.index[-1])]) :
            tks.at[sy,timefr]=-1
            
            ord.at[sy,'Signal']=-1
            ord.at[sy,'SignalTime']=str(time.ctime())
            #print('                                  держим продажи')
            return True
    else:  
            tks.at[sy,timefr]=0
            
            ord.at[sy,'Signal']=0
            ord.at[sy,'SignalTime']=str(time.ctime())        
            #print('                                                          Шляпа какая-то')
    return True



def filtrVol (df):
    dfv=df
    

def zapisHistory(df,tf,simb):
    df.to_csv(Hist+simb+tf+'Obrabotka.csv')

def zapisHistory1(df,tf,simb):
   
        df1=pd.read_csv(Hist+simb+tf+'Obrabotka.csv')
        df4=df1.append(df)
        df4.to_csv(Hist+simb+tf+'Obrabotka.csv')

def tfizm5(df,n):
    df1=pd.DataFrame(columns = ['open_time','O','H','L','C','V','close_time'])
    i=0
    i1=0
    t=0
    while i <len(df.index):
        if (df.index[i].hour==0 or df.index[i].hour==4 or df.index[i].hour==8 or df.index[i].hour==12 or df.index[i].hour==16 or df.index[i].hour==20) and df.index[i].minute==0 and t==0:
            t=1
        if t==0:
            i+=1
        if t==1:
            df1.at[i1,'open_time']=df.index[i-n]
            #df.loc[df.index[4:7],'H']
            df1.at[i1,'H']=df.loc[df.index[i-n:i],'H'].max()
            df1.at[i1,'L']=df.loc[df.index[i-n:i],'L'].min()
            df1.at[i1,'O']=df.at[df.index[i-n],'O']
            df1.at[i1,'C']=df.at[df.index[i-1],'C']
            
            df1.at[i1,'close_time']=df.at[df.index[i-1],'close_time']
            df1.at[i1,'V']=df.loc[df.index[i-n:i],'V'].sum()
            i1+=1
            i+=n
    df1.open_time= pd.DatetimeIndex(df1['open_time'])
    df1=df1. set_index ('open_time')
    return(df1)

def reducir():
        wpos=''
        status = client.futures_account()
        positions1=pd.DataFrame(status['positions'])
        for pos in tks.index:
            symbPos=tks.symbol[pos]
            order1 = positions1[positions1['symbol']==symbPos]['positionAmt'].astype(float).tolist()[0]
            finrez = positions1[positions1['symbol']==symbPos]['unrealizedProfit'].astype(float).tolist()[0]
            notional = abs(positions1[positions1['symbol']==symbPos]['notional'].astype(float).tolist()[0])
            #leverage = int(positions[positions['symbol']==symbol]['leverage'])
            ord.at[pos,'Order'] = order1
            # Reduciruem loss and profit
            if notional==0 or order1==0:
                ord.at[pos,'OrderPrice']  =0
                ord.at[pos,'FinRez']=0
                ord.at[pos,'MaxFinRez']=0       
         
         
            if (notional)>0:
            
                if finrez>ord.at[pos,'MaxFinRez'] and (finrez)>0.2 :
                    ord.at[pos,'MaxFinRez']=finrez     
                    ord.at[pos,'FinRez'] = finrez
                    ord.at[pos,'OrderPrice']  = positions1[positions1['symbol']==symbPos]['entryPrice']  
                    
                wpos=wpos+(symbPos+' order '+str(order1)+' rez'+str(finrez)+ ' summa '+str(positions1[positions1['symbol']==symbPos]['notional'].astype(float).tolist()[0])+ '    проц '+ str(round((100*finrez)/notional,2)))
                #Take profit       
                if (  finrez>0.7) :
                    if order1>0:
                        open_position(symbPos,'short',abs(order1))
                        print(str(finrez)+'  take profit 1.3  '+symbPos)                   
                    elif order1<0:
                        open_position(symbPos,'long',abs(order1))
                        print(str(finrez)+'  take profit 1.3   '+symbPos)
                   # Reduciruem
                elif (finrez>0.2 or finrez<-0.07) and (abs(order1))>float(tks.at[pos,'minOrd']) :
                    if order1>0:
                        open_position(symbPos,'short',(float(tks.at[pos,'minOrd'])))
                        print(str(finrez)+'  редуцируем по инструменту>0.03 or<-0,05 '+symbPos)                   
                    elif order1<0:
                        open_position(symbPos,'long',(float(tks.at[pos,'minOrd'])))
                        print(str(finrez)+'  редуцируем по инструменту>0.03 or<-0,05 '+symbPos)
                      #Profitnu v bezubitok
                elif finrez<(ord.at[pos,'MaxFinRez'])/4 and ord.at[pos,'MaxFinRez']!=0:
                    if order1>0:
                        open_position(symbPos,'short',abs(order1))
                        print(str(finrez)+'  ,bezubitok '+symbPos+str(order1))                 
                    elif order1<0:
                        open_position(symbPos,'long',abs(order1))
                        print(str(finrez)+'  bezubitok '+symbPos+str(order1))                        
                                        
                  #stop loss
                elif finrez<-0.2:
                    if order1>0:
                        open_position(symbPos,'short',abs(order1))
                        print(str(finrez)+'  stop loss <-0.5 '+symbPos+str(order1))                 
                    elif order1<0:
                        open_position(symbPos,'long',abs(order1))
                        print(str(finrez)+'   stop loss <-0.5 '+symbPos+str(order1))
        ord.to_csv(path+'OrdersFutures.csv')
        return(wpos)
                  
        #time.sleep(1)    

def chartTest(data,sym='',napr=''):
   # Create figure with secondary y-axis
  fig = make_subplots(specs=[[{"secondary_y": True}]])

  # include candlestick with rangeselector
  fig.add_trace(go.Candlestick(x=data.open_time, 
                                 low = data.low,
                                 high = data.high,
                                 close = data.close,
                                 open = data.open), secondary_y=True)
  # include a go.Bar trace for volumes
  fig.add_trace(go.Bar(x=data.open_time, y=data.volume,name='volume'), secondary_y=False)
  fig.add_trace(go.Scatter(x=data.open_time, y=data.sig4h,name='sig4h'), secondary_y=False)
  fig.add_trace(go.Scatter(x=data.open_time, y=data.sig1h,name='sig1h'), secondary_y=False)
  fig.add_trace(go.Scatter(x=data.open_time, y=data.sigvh,name='sigvh'), secondary_y=False)
  fig.add_trace(go.Scatter(x=data.open_time, y=data.vhod5m,name='vhod5m'), secondary_y=False)
  fig.add_trace(go.Scatter(x=data.open_time, y=data.vhod15m,name='vhod15m'), secondary_y=False)
  fig.add_trace(go.Scatter(x=data.open_time, y=data.vhod1h,name='vhod1h'), secondary_y=False)
  fig.add_trace(go.Scatter(x=data.open_time, y=data.vhod4h,name='vhod4h'), secondary_y=False)
  fig.add_trace(go.Scatter(x=data.open_time, y=data.prsell,name='prsell'), secondary_y=False)
  fig.add_trace(go.Scatter(x=data.open_time, y=data.profit,name='profit'), secondary_y=False)
  fig.add_trace(go.Scatter(x=data.open_time, y=data.otkr,name='otkr'), secondary_y=False)
  fig.add_trace(go.Scatter(x=data.open_time, y=data.zakr,name='zakr'), secondary_y=False)
  fig.add_trace(go.Scatter(x=data.open_time, y=data.perev,name='perev'), secondary_y=False)
  fig.add_trace(go.Scatter(x=data.open_time, y=data.prib,name='prib'), secondary_y=False)
  fig.add_trace(go.Scatter(x=data.open_time, y=data.comiss,name='comiss'), secondary_y=False)

  fig.layout.yaxis2.showgrid=False
  fig.show()    
  #['sig4h', 'sig1h','sigvh','vhod5m','vhod15m','vhod1h','vhod4h','prsell','comiss','profit','otkr','zakr','perev','prib'])])
  #graph = go.Figure()
  #Adding Trace
  graph = make_subplots(specs=[[{"secondary_y": True}]])
  graph.add_trace(go.Candlestick(x=data.open_time,
                     open=data.open,
                     high=data.high,
                     low=data.low,
                     close=data.close, name = sym), secondary_y=True) 
  graph.add_trace(go.Scatter(x=data.open_time, y=data['DeltCOEXPMA'],name='DeltCOEXPMA'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.open_time, y=data['DeltHLMA'],name='DeltHLMA'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.open_time, y=data['DeltHL'],name='DeltHL'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.open_time, y=data['DeltCO'],name='DeltCO'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.open_time, y=data['DeltHO'],name='DeltHO'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.open_time, y=data['DeltOL'],name='DeltOL'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.open_time, y=(data['DeltCL']),name='DeltCL'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.open_time, y=(data['DeltHC']),name='DeltHC'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.open_time, y=(data['DeltHLCO']),name='DeltHLCO'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.open_time, y=(data['DeltHLCOMA']),name='DeltHLCOMA'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.open_time, y=(data['OtnHLCO']),name='OtnHLCO'), secondary_y=False)
 


  
  graph.add_trace(go.Scatter(x=data.open_time, y=(data['DeltHC']+data['DeltCL']),name='DeltHC + DeltCL'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.open_time, y=((data['DeltHO']+data['DeltOL'])),name='DeltHO+DeltOL'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.open_time, y=(data['DeltHO']+data['DeltCL']),name='DeltHO + DeltCL'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.open_time, y=((data['DeltHC']+data['DeltOL'])),name='DeltHC+DeltOL'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.open_time, y=data['SummDelt'],name='SummDelt'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.open_time, y=data['SummMedian'],name='SummMedian'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.open_time, y=(data['ClMed']),name='Close-mediana'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.open_time, y=(data['MedOp']),name='mediana-open'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.open_time, y=data['DeltCL_HC'],name='дельта по клоузу'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.open_time, y=data['DeltHO_OL'],name='дельта по опену'), secondary_y=False)
  
  
  graph.add_hline(0)
  graph.layout.yaxis2.showgrid=False
  #Adding updates
  graph.update_xaxes(
  rangeslider_visible=True,title = 'Показатели')
  #Asking Plotly to show
  graph.show()

def chart(data,sym='',napr=''):
   # Create figure with secondary y-axis
  fig = make_subplots(specs=[[{"secondary_y": True}]])

  # include candlestick with rangeselector
  fig.add_trace(go.Candlestick(x=data.index,
                                 low = data.L,
                                 high = data.H,
                                 close = data.C,
                                 open = data.O), secondary_y=True)
  # include a go.Bar trace for volumes
  #fig.add_trace(go.Bar(x=data.index, y=data.SL,name='SL'), secondary_y=False)
 
  fig.add_trace(go.Scatter(x=data.index, y=data.XCO,name='XCO'), secondary_y=False)
  fig.add_trace(go.Scatter(x=data.index, y=data.XCOMA,name='XCOMA'), secondary_y=False)

  fig.layout.yaxis2.showgrid=False
  fig.show()    

  #graph = go.Figure()
  #Adding Trace
  graph = make_subplots(specs=[[{"secondary_y": True}]])
  graph.add_trace(go.Candlestick(x=data.index,
                     open=data.O,
                     high=data.H,
                     low=data.L,
                     close=data.C, name = sym), secondary_y=True)
  graph.add_trace(go.Scatter(x=data.index, y=data['IzmCO'],name='IzmCO'), secondary_y=False)
  #graph.add_trace(go.Scatter(x=data.index, y=data['short1'],name='short1'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.index, y=data['VBvverh'],name='VBvverh'), secondary_y=False)
  #graph.add_trace(go.Scatter(x=data.index, y=data['long1'],name='long1'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.index, y=data['VBniz'],name='VBniz'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.index, y=data['Mhvost'],name='Mhvost'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.index, y=data['hvost'],name='hvost'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.index, y=data['SLong'],name='SLong'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.index, y=data['SShort'],name='SShort'), secondary_y=False) 
  graph.add_trace(go.Scatter(x=data.index, y=data['DeltCOMA'],name='DeltCOMA'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.index, y=data['DeltHLMA'],name='DeltHLMA'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.index, y=data['DeltHL'],name='DeltHL'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.index, y=data['DeltCO'],name='DeltCO'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.index, y=data['DeltHO'],name='DeltHO'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.index, y=data['DeltOL'],name='DeltOL'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.index, y=(data['DeltCL']),name='DeltCL'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.index, y=(data['DeltHC']),name='DeltHC'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.index, y=data.SignShort,name='SignShort'), secondary_y=False)
  graph.add_trace(go.Scatter(x=data.index, y=data.SignLong,name='SignLong'), secondary_y=False)  
  
  #graph.add_hline(0)
  graph.layout.yaxis2.showgrid=False
  #Adding updates
  graph.update_xaxes(
  rangeslider_visible=True,title = 'Показатели')
  #Asking Plotly to show
  graph.show()

def podgData(df,n=1):
    #df1=pd.read_csv('/content/drive/MyDrive/history/'+symb+timefr+'Obrabotka.csv')
    #df2 =get_futures_klines(symb,timefr,10) 
     
    #ma=max(df['H'])
    #vo=max(df['V'])
    #op=df.O[1]
    #df.O=df.O/ma*100
    #df.C=df.C/ma*100
    #df.H=df.H/ma*100
    #df.L=df.L/ma*100
    #op1=df.close.mean()
    #df.open=df.open-op1
    #df.close=df.close-op1
    #df.high=df.high-op1
    #df.low=df.low-op1
    #df.volume=df.volume/vo
    df=pd.concat([df, pd.DataFrame(columns = ['hvost','Mhvost','DeltHL','DeltCO','DeltHC','DeltHO','DeltCL','DeltOL','DeltHLMA',
                                              'volumMA','SignShort','SignLong','DeltCOMA','DinLong','DinShort','DeltaCO1',
                                              'long1','long2','short1','short2','IzmCO','VBvverh','VBniz'  ])])
    
    df['hvost'] =((df.H-df.L)-abs(df.C-df.O))
    
    
    df['DeltHLMA'] =(df.DeltHL).mean()
    
    df.IzmCO=((df.DeltCO/df.DeltCO.shift())**2)*(df.DeltCO)
    
    #df['DeltCOEXP'] =(df.C-df.O).rolling(5).mean()
    #df['DeltCOEXPMA'] =(df.DeltCOEXP).rolling(5).mean()
    df['DeltHL'] =(df.H-df.L).rolling(n).sum()
    df['DeltCO'] =(df.C-df.O).rolling(n).sum()
    df['DeltHO'] =(df.H-df.O).rolling(n).sum()
    df['DeltCL'] =(df.C-df.L).rolling(n).sum()
    df['DeltOL'] =(df.O-df.L).rolling(n).sum()
    df.fillna(0)
    df.DeltHL=df.DeltHL/max(df['DeltHL'])
        
    df.DeltCO1 =(df.DeltCO>0)*1
    df.DeltCO=abs(df.DeltCO)/max(df['DeltCO'])
    
    df['Mhvost']=(df.DeltCO)/(df.hvost)
    df['Mhvost3']=df['Mhvost']*df['Mhvost']*df['Mhvost']
    df['DeltCOMA'] =df['DeltCO']*df['DeltCO']*df['DeltCO']
    df['Mhvost']=(df.DeltCO)/(df.hvost)
    df['DeltHC'] =(df.H-df.C).rolling(n).sum()
    df.DeltHC=df.DeltHC/max(df['DeltHC'])
    
    
    df.DeltHO=df.DeltHO/max(df['DeltHO'])
    
    
    df.DeltCL=df.DeltCL/max(df['DeltCL'])
    
    
    df.DeltOL=df.DeltOL/max(df['DeltOL'])
    df.fillna(0)

    df.VBvverh= ((df.C.shift(3)>df.O.shift(3))&(df.H.shift(3)>=df.C.shift(2))&(
        df.L.shift(3)<=df.C.shift(2))&(df.H.shift(3)>=df.C.shift(1))&(df.L.shift(3)<=df.C.shift(1)))*1
    
    df.VBniz= ((df.C.shift(3)<df.O.shift(3))&(df.H.shift(3)>=df.C.shift(2))&(
        df.L.shift(3)<=df.C.shift(2))&(df.H.shift(3)>=df.C.shift(1))&(df.L.shift(3)<=df.C.shift(1)))*1
    
    df['XCO'] =(df['DeltHO'] -df['DeltOL']+df['DeltCL']-df['DeltOL']+df['DeltHO']-(df['DeltHC']))+df['DeltCO']
    df.fillna(0)
    #df['SS'] =(df['DeltOL'] +df['DeltHC'])+df['DeltCO']
    #df['X'] =df['DeltHO'] +df['DeltCL']-df['DeltHC']-df['DeltOL']
    #df['XCO'] =df['X']*abs(df['DeltCO'])
    df['XCOMA'] =df['XCO']*df['XCO']*df['XCO']
    #df.long1= (df.DeltHO>df.DeltCL)&(df.DeltCL> df.DeltHC)&(df.DeltHC>df.DeltOL)& (
     #   df.slopeHO>df.slopeCL)&(df.slopeCL>0)& (0>df.slopeHC)&(df.slopeHC> df.slopeOL) 
    df.long2= (df.DeltOL<df.DeltOL.shift())&(df.DeltHO>df.DeltHO.shift())& (df.DeltCL>df.DeltCL.shift())&(df.DeltCL>df.DeltHC)& (df.DeltHO>df.DeltOL)&(df.DeltHO>df.DeltHC)&(df.DeltCL>df.DeltOL)
    #df.short1= (df.DeltHO<df.DeltCL)&(df.DeltCL< df.DeltHC)&(df.DeltHC<df.DeltOL)& (
    #    df.slopeHO<df.slopeCL)&(df.slopeCL<0)& (0<df.slopeHC)&(df.slopeHC< df.slopeOL)
    df.short2= (df.DeltOL>df.DeltOL.shift())& (df.DeltHO<df.DeltHO.shift())& (df.DeltCL<df.DeltHC)& (df.DeltHO<df.DeltOL) &( df.DeltHO<df.DeltHC)&(df.DeltCL<df.DeltOL) 
 
    

    
    #DinLong=(df.DeltCL>df.DeltCL.shift())&(df.DeltOL<df.DeltOL.shift())
    #DinShort =(df.DeltHO<df.DeltHO.shift())&(df.DeltHC>df.DeltHC.shift())


    df['SLong'] =((df.DeltCL-df.DeltOL)+(df.DeltHO-df.DeltHC)+(df.DeltHO-df.DeltOL))
    df['SShort'] =((-df.DeltHO+df.DeltHC)+(-df.DeltCL+df.DeltOL)+(-df.DeltHO+df.DeltOL))
    df['SignShort'] =1*((df.DeltHO<df.DeltHC)&(df.DeltCL<df.DeltOL)&(df.DeltHO<df.DeltOL))
    df['SignLong'] =1*((df.DeltCL>df.DeltOL)&(df.DeltHO>df.DeltHC)&(df.DeltHO>df.DeltOL))
    df['volumMA']= df.V.rolling(n).sum()

  
    df.fillna(0)
   
    return(df)  



def podgData2(df,n=1):
    #df1=pd.read_csv('/content/drive/MyDrive/history/'+symb+timefr+'Obrabotka.csv')
    #df2 =get_futures_klines(symb,timefr,10) 
     
    ma=max(df['H'])
    vo=max(df['V'])
    op=df.O[1]
    df.O=df.O/ma
    df.C=df.C/ma
    df.H=df.H/ma
    df.L=df.L/ma
    #op1=df.close.mean()
    #df.open=df.open-op1
    #df.close=df.close-op1
    #df.high=df.high-op1
    #df.low=df.low-op1
    df.V=df.V/vo
    df=pd.concat([df, pd.DataFrame(columns = ['DeltHL','DeltHC','DeltHO','DeltCL',
                                              'DeltOL' ])])
    
    #df['hvost'] =((df.H-df.L)-abs(df.C-df.O))
    
    df['DeltHL'] =(df.H-df.L).rolling(n).sum()
    df.DeltHL=df.DeltHL/max(df['DeltHL'])
    
    df['DeltCO'] =(df.C-df.O).rolling(n).sum()
    #df.DeltCO1 =(df.DeltCO>0)*1
    #df.DeltCO=abs(df.DeltCO)/max(df['DeltCO'])
    
    #df['Mhvost']=(df.DeltCO)/(df.hvost)
    df['DeltHC'] =(df.H-df.C).rolling(n).sum()
    df.DeltHC=df.DeltHC/max(df['DeltHC'])
    
    df['DeltHO'] =(df.H-df.O).rolling(n).sum()
    df.DeltHO=df.DeltHO/max(df['DeltHO'])
    
    df['DeltCL'] =(df.C-df.L).rolling(n).sum()
    df.DeltCL=df.DeltCL/max(df['DeltCL'])
    
    df['DeltOL'] =(df.O-df.L).rolling(n).sum()
    df.DeltOL=df.DeltOL/max(df['DeltOL'])
    df=df.drop(['close_time'], axis=1)
    df.fillna(0)#,'O','C','H','L','V'

   
    return(df)  

def bactest(ohlc,n):
    #interact(=3);

    #ohlc=podgData(ohlc)

    sb = ohlc.VBvverh#SLong#long2
    ss = ohlc.VBniz#SShort#2
    
    buy = cover = sb==1#(sb>ss)&(sb>sb.shift())
    sell = short = ss==1#(sb<ss)&(ss>ss.shift())

#print('>  buy MA\n%s\n' % sb.tail(20))
#print('>  short\n%s\n' % ss.tail(20))
#print('>  Buy/Cover signals\n%s\n' % buy.tail())
#print('>  Short/Sell signals\n%s\n' % sell.tail())

    bt = pybacktest.Backtest(locals(), 'HHH')

#print(list(filter(lambda x: not x.startswith('_'), dir(bt))))
#print('\n>  bt.signals\n%s' % bt.signals.tail())
#print('\n>  bt.trades\n%s' % bt.trades.tail())
#print('\n>  bt.positions\n%s' % bt.positions.tail())
#print('\n>  bt.equity\n%s' % bt.equity.tail())
#print('\n>  bt.trade_price\n%s' % bt.trade_price.tail())
    #sum(bt.equity.tail(3)) 
    #bt.equity.tail(15),
    #bt.summary()
    if sum(bt.equity) >0:
        return sum(bt.equity.tail(n))
    else: return sum(bt.equity.tail(n))
    
def Signal(df):
    ohlc=df
    sb = ohlc.H#long2
    ss = ohlc.L#2
    

    long =(ss>ss.shift())&(ohlc.O<ohlc.C)#(sb>ss)&(sb>sb.shift())
    short =(sb<sb.shift())&(ohlc.O<ohlc.C)# (sb<ss)&(ss>ss.shift())
    
   
    
    if   long[-1]==1 :
        return 1
    elif  short[-1]==1  :
        return -1 
    else:
        return 0
    


In [ ]:
ohlc=df2
sb = ohlc.long2
ss = ohlc.short2
    

long= (sb==1)&(sb.shift()==0)
short =  (ss==1)&(ss.shift()==0)

# Test


In [ ]:
corr_matrix = df777[list(df777.columns)].corr()
sns.set(rc = {'figure.figsize':(30,15)})
sns.heatmap(corr_matrix, annot=True)
pass

In [ ]:
apiKey='BK53YCTI6Q5VENNY'

interval_var = '5min'
symbol='ETH'

path='https://www.alphavantage.co/query?function=CRYPTO_INTRADAY&symbol='+symbol+'&market=USD&interval='+interval_var+'&apikey='+apiKey+'&datatype=csv&outputsize=full'
df = pd.read_csv(path)





In [21]:
df1 =get_futures_klines('AAVEUSDT','1m',1500)
#df1.to_csv('AAVEUSDT'+'5m'+'.csv')


In [ ]:
df5=podgData(df1,1)

In [ ]:
chart(df5)

In [20]:
df2=podgData2(df1,1)

In [12]:
df2

,O,H,L,C,V,close_time
open_time,,,,,,
2022-03-17 21:45:00,0.558314,0.559806,0.556899,0.558773,0.016711,2022-03-17 21:59:59.999
2022-03-17 22:00:00,0.558735,0.563784,0.558620,0.560571,0.051989,2022-03-17 22:14:59.999
2022-03-17 22:15:00,0.560532,0.561909,0.557817,0.558773,0.021623,2022-03-17 22:29:59.999
2022-03-17 22:30:00,0.559079,0.562483,0.558390,0.561871,0.026876,2022-03-17 22:44:59.999
2022-03-17 22:45:00,0.561833,0.564128,0.561221,0.563822,0.024934,2022-03-17 22:59:59.999
...,...,...,...,...,...,...
2022-04-02 11:30:00,0.922580,0.928853,0.922235,0.926022,0.020476,2022-04-02 11:44:59.999
2022-04-02 11:45:00,0.926022,0.927973,0.924416,0.927973,0.017603,2022-04-02 11:59:59.999
2022-04-02 12:00:00,0.927820,0.943121,0.927132,0.936886,0.080968,2022-04-02 12:14:59.999


In [22]:
scaler=StandardScaler()
df2T=scaler.fit_transform(df2)

In [25]:


KM=KMeans(n_clusters=10,  init='k-means++', n_init=10, max_iter=300, tol=0.0001, 
          verbose=0, random_state=None, copy_x=True, algorithm='auto')
AC=AgglomerativeClustering(n_clusters=10,  affinity='euclidean', memory=None, 
                           connectivity=None, compute_full_tree='auto', linkage='ward',
                           distance_threshold=None, compute_distances=False)

DB=DBSCAN(eps=0.5, min_samples=50, metric='euclidean', metric_params=None,
          algorithm='auto', leaf_size=30, p=None, n_jobs=None)

AP=AffinityPropagation( damping=0.5, max_iter=200, convergence_iter=15,
                    copy=True, preference=None, affinity='euclidean', verbose=False, random_state=None)
MS=MeanShift( bandwidth=None, seeds=None, bin_seeding=False, min_bin_freq=1, cluster_all=True, n_jobs=None, max_iter=300)

CC=SpectralClustering ( n_clusters = 8  , eigen_solver = None , n_components = None , random_state = None,
                       n_init = 10 , gamma = 1.0 , affinity = 'rbf' , n_neighbors = 10 , eigen_tol = 0.0 , 
                       assign_labels = 'kmeans' , coef0 = 1 , kernel_params = None , n_jobs = None, 
                       verbose = False )
B=Birch( threshold=0.5, branching_factor=50, n_clusters=3, compute_labels=True, copy=True)

OPT=OPTICS ( min_samples=5, max_eps=np.inf, metric='minkowski', p=2, metric_params=None, cluster_method='xi', eps=None, xi=0.05, predecessor_correction=True, min_cluster_size=None,
           algorithm='auto', leaf_size=30, memory=None, n_jobs=None)




df3['clasKMeans1']=KM.fit_predict(df2)
df3['clasAC1']=AC.fit_predict(df2)
df3['clasDB1']=DB.fit_predict(df2)
df3['clasAP1']=AP.fit_predict(df2)
df3['clasMeanShift1']=MeanShift.fit_predict(df2)
df3['SpectralClustering1']=SpectralClustering.fit_predict(df2)
df3['clasBirch1']=Birch.fit_predict(df2)
df3['clasOPTICS1']=OPTICS.fit_predict(df2)


df3['clasKMeans']=KM.fit_predict(df2T)
df3['clasAC']=AC.fit_predict(df2T)
df3['clasDB']=DB.fit_predict(df2T)
df3['clasAP']=AP.fit_predict(df2T)
df3['clasMeanShift']=MeanShift.fit_predict(df2T,df3.clasAP)
df3['SpectralClustering']=SpectralClustering.fit_predict(df2T)
df3['clasBirch']=Birch.fit_predict(df2T)
df3['clasOPTICS']=OPTICS.fit_predict(df2T)



TypeError: ClusterMixin.fit_predict() missing 1 required positional argument: 'X'

In [24]:
df3

,O,H,L,C,V,DeltHL,DeltHC,DeltHO,DeltCL,DeltOL,DeltCO,clasKMeans1,clasAC1,clasDB1,clasAP1
2022-03-26 16:30:00,0.677016,0.678913,0.675779,0.678212,0.007478,0.055434,0.024182,0.040386,0.055295,0.022026,0.001196,0,3,0,0
2022-03-26 16:45:00,0.678212,0.680068,0.676728,0.678130,0.017732,0.059081,0.066856,0.039508,0.031865,0.026432,-0.000082,0,3,0,0
2022-03-26 17:00:00,0.678130,0.679944,0.674872,0.675243,0.016280,0.089716,0.162162,0.038630,0.008435,0.058003,-0.002886,6,3,0,1
2022-03-26 17:15:00,0.675202,0.676357,0.673347,0.675697,0.010196,0.053246,0.022760,0.024583,0.053421,0.033040,0.000495,0,3,0,0
2022-03-26 17:30:00,0.675903,0.677222,0.673759,0.673841,0.007799,0.061269,0.116643,0.028095,0.001874,0.038179,-0.002062,0,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-31 20:15:00,0.864753,0.867928,0.860713,0.865083,0.031137,0.127644,0.098151,0.067603,0.099344,0.071953,0.000330,1,1,0,13
2022-03-31 20:30:00,0.865248,0.871103,0.865207,0.867310,0.027590,0.104303,0.130868,0.124671,0.047798,0.000734,0.002062,1,1,0,19
2022-03-31 20:45:00,0.866939,0.875557,0.866032,0.872011,0.031927,0.168490,0.122333,0.183494,0.135895,0.016153,0.005072,1,1,0,20
2022-03-31 21:00:00,0.871969,0.872011,0.860341,0.862609,0.029371,0.206419,0.324324,0.000878,0.051546,0.207048,-0.009360,8,2,0,12


In [ ]:
df3.hist()

In [ ]:
df3.plot(figsize=(20,20),subplots=True)

In [ ]:
corr=df3.corr()

In [ ]:
df3.clasAP.value_counts()


In [ ]:
corel=df2.corr(method='spearman', min_periods=5)

In [ ]:
sns.heatmap(corel, annot=True)

In [ ]:
sns.heatmap(corel, annot=True)

In [ ]:
sns.heatmap(corel, annot=True)

In [ ]:
#corr_matrix = corel[list(df777.columns)].corr()
#sns.set(rc = {'figure.figsize':(20,10)})
sns.heatmap(corel, annot=True)


In [ ]:
sns.heatmap(corel, annot=True)

In [26]:
p=0
def f(x,n):
    #interact(=3);
    #df2=podgData(df1,n)
    ohlc=df1
    
    sb = ohlc.H#long2
    ss = ohlc.L#2
    
    if ohlc.V>ohlc.V.roling(n).mean():
        buy = cover =(ss>ss.shift())&(ohlc.O<ohlc.C)#(sb>ss)&(sb>sb.shift())
        sell = short =(sb<sb.shift())&(ohlc.O<ohlc.C)# (sb<ss)&(ss>ss.shift())

    #print('>  buy MA\n%s\n' % sb.tail(20))
    #print('>  short\n%s\n' % ss.tail(20))
    #print('>  Buy/Cover signals\n%s\n' % buy.tail())
    #print('>  Short/Sell signals\n%s\n' % sell.tail())

    bt = pybacktest.Backtest(locals(), 'HHH')

    #print(list(filter(lambda x: not x.startswith('_'), dir(bt))))
    #print('\n>  bt.signals\n%s' % bt.signals.tail())
    #print('\n>  bt.trades\n%s' % bt.trades.tail())
    #print('\n>  bt.positions\n%s' % bt.positions.tail())
    #print('\n>  bt.equity\n%s' % bt.equity.tail())
    #print('\n>  bt.trade_price\n%s' % bt.trade_price.tail())
    print(sum(bt.equity)) 
    #print((bt.equity.tail(10)))
    print(sum(bt.equity.tail(5)))
    bt.summary()
    #chart(df2.tail(20))
    return sum(bt.equity.tail(x)) ,bt.equity.tail(15)


In [33]:
df1.V.value_counts()

1646.8    2
261.7     2
381.5     2
407.7     2
541.0     2
         ..
7119.6    1
6179.4    1
5525.4    1
3405.3    1
634.8     1
Name: V, Length: 1464, dtype: int64

In [36]:
interact(f, x=widgets.IntSlider(min=1, max=30, step=1, value=4),n=widgets.IntSlider(min=1, max=10, step=1, value=1));

interactive(children=(IntSlider(value=4, description='x', max=30, min=1), IntSlider(value=1, description='n', …

# график



In [ ]:
chart(df2.tail(30))

In [ ]:
chart(df2)

In [ ]:
chart(df2.head(1000))

# vectorbt


In [ ]:
!pip install pybacktest

In [ ]:
chart(df1)

In [ ]:
import vectorbt as vbt

price = vbt.YFData.download('BTC-USD').get('Close')

pf = vbt.Portfolio.from_holding(price, init_cash=100)
pf.total_profit()


In [ ]:
fast_ma = vbt.MA.run(price, 10)
slow_ma = vbt.MA.run(price, 50)
entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

pf = vbt.Portfolio.from_signals(price, entries, exits, init_cash=100)
pf.total_profit()

# Перезапуск таблицы символов и обновление инфы об ордерах

In [34]:
#ПЕРЕЗАПУСК и СБРОС информации о инструментах

GpathIm ='/content/drive/MyDrive/images/'
GpathH='/content/drive/MyDrive/history/'
Gpath='/content/drive/MyDrive/Bot/'
pathH='history/'
pathI='images/'
path0=''

GC=0
if GC==1:
    Im=GpathIm
    Hist=GpathH
    path=Gpath
else:
    Im=pathI
    Hist=pathH
    path=path0


future= True
if future==True:
    tk=pd.read_csv(path+'symbFuture.csv',delimiter=';') 
else:
    tk=pd.read_csv(path+'symbol.csv',delimiter=';') 
    
tks=pd.concat([tk, pd.DataFrame(columns = ['vhod1m','vhod5m','vhod15m','vhod1h','vhod4h','time'])])
tks[['vhod1m','vhod5m','vhod15m','vhod1h','vhod4h']] = 0.0
tks.to_csv(path+'SignalsymbFuture.csv')   
ord=pd.concat([tk, pd.DataFrame(columns = ['Signal','SignalTime', 'SignalPrice','Order','OrderTime',
                                               'OrderPrice','FinRez','FinRezTime'])])
ord[['Signal', 'SignalPrice','Order','OrderPrice','FinRez','MaxFinRez']] = 0.0

Testord=pd.concat([tk, pd.DataFrame(columns = ['Signal','SignalTime', 'SignalPrice','Order','OrderTime',
                                           'OrderPrice','FinRez','FinRezTime'])])
Testord[['Signal', 'SignalPrice','Order','OrderPrice','FinRez','MaxFinRez']] = 0.0

# ОСНОВНОЕ проверка и торговля


In [35]:

#N NEW 27.02.2022
#                         ОСНОВНОЕ ПРОВЕРКА И ТОРГОВЛЯ
#

wposition =widgets.Textarea(disabled=True)
wsymb =widgets.Textarea(disabled=True)
wprofit=widgets.Text(disabled=True)
ww=widgets.Textarea(disabled=True)
display(wprofit)
display(wsymb,ww)
display(wposition)


wstop=widgets.ToggleButton(value=True, description='Stop',  disabled=True,
                           button_style='', # 'success', 'info', 'warning', 'danger' or ''
                           tooltip='Description',
                           icon='check' # (FontAwesome names without the `fa-` prefix)
                          )
p=True
display(wstop)
while p==True:
    
    for sy in tks.loc[:77].index:
        if sy%10==0 and future==True:
            wposition.value=reducir()
        
        status = client.futures_account()
        positions=pd.DataFrame(status['positions'])     
        vhod1h=0
        vhod5m=0
        vhod4h=0 
        rol=0
        #p=wstop
        symb=tks.symbol[sy]
        wsymb.value=('              '+ symb+'     ')
        if future==True:
            order = positions[positions['symbol']==symb]['positionAmt'].astype(float).tolist()[0]
            finrez=positions[positions['symbol']==symb]['unrealizedProfit'].astype(float).tolist()[0]
            notional = abs(positions[positions['symbol']==symb]['notional'].astype(float).tolist()[0])
            ord.at[sy,'OrderPrice']  = positions[positions['symbol']==symb]['entryPrice']
            profit = float(status['totalUnrealizedProfit'])
            balance = round(float(status['totalWalletBalance']),2)
            ord.at[sy,'OrderPrice']  = positions[positions['symbol']==symb]['entryPrice']
            ord.at[sy,'Order'] = order
            ord.at[sy,'FinRez'] = finrez   
            wprofit.value=(' profit= '+str(profit)+'  balance= '+str(balance))     
        
        #analiz tfr
        rol1=0  
        tfr='1m'
        try:
            if future==True:
                df=get_futures_klines(symb,tfr,1500)
            else:
                df=get_spot_klines(symb,tfr,1500)
        except:
            print('Исключение')
            time.sleep(5)
            try:
                if future==True:
                    df=get_futures_klines(symb,tfr,1500)
                else:
                    df=get_spot_klines(symb,tfr,1500)
            except:
                print('Исключение')
                time.sleep(10)
                df=get_futures_klines(symb,tfr,1500)
        df1m=podgData(df)
        df5m=tfizm5(df,5)
        df5m=podgData(df5m)
        rol2=bactest(df5m,3)
        rol1=bactest(df1m,3)
        #print(rol1,rol2,symb)
            
        if  (rol1>0 and rol2>0 )or ord.at[sy,'Order']!=0:#torguem ili net na m5
            try:
                df4h=tfizm5(df,240)
                df4h=podgData(df4h,4)
                vhod4h=Signal(df4h)
                tks.at[sy,'vhod4h']=vhod4h
            
                df1h=tfizm5(df,60)
                df1h=podgData(df1h.tail(40))
                #if bactest(df1h,1)>0:
                vhod1h=Signal(df1h)
                tks.at[sy,'vhod1h']=vhod1h
            except:
                print('Исключение2')
                vhod4h=0
                vhod1h=0
               
            #df1m=podgData(df.tail(500))
            vhod1m=Signal(df1m)
            tks.at[sy,'vhod1m']=vhod1m
            
            
            #if bactest(df5m,1)>0:
            vhod5m=Signal(df5m) 
            tks.at[sy,'vhod5m']=vhod5m
            
            
            df15m=tfizm5(df,15)
            df15m=podgData(df15m)
            #if bactest(df5m,1)>0:
            vhod15m=Signal(df15m)
            tks.at[sy,'vhod15m']=vhod15m
            tks.at[sy,'time']= str(time.ctime())
          
            if ord.at[sy,'Order']!=0:
                print(str(time.ctime()),symb,vhod1m,vhod5m,vhod15m,vhod1h,vhod4h,df1m.C[df1m.index[-1]])
            
            ww.value=(' '+str(vhod1m)+'  '+str(vhod5m)+'  '+str(vhod15m)+'  '+str(vhod1h)+'  '+str(vhod4h))            
            i=-1
            
            if  ord.at[sy,'Order']==0 and (vhod1m==1 and vhod5m==1 and vhod15m!=-1 )and vhod1h!=-1   and vhod4h!=-1 :
                open_position(symb,'long',2*abs(float(tks.at[sy,'minOrd'])))
                ord.at[sy,'Order']=2*float(tks.at[sy,'minOrd'])
                ord.at[sy,'OrderTime']=str(time.ctime())
                print(symb+' Зашли BUY '+str(get_symbol_price(symb))+' Order= '+str(ord.at[sy,'Order'])+'  Signal= '+ str(ord.at[sy,'Signal'])+str(time.ctime()))
                print(' '+str(vhod1m)+'  '+str(vhod5m)+'  '+str(vhod15m)+'  '+str(vhod1h)+'  '+str(vhod4h)) 
              #sendImage(symb+timefr)
                              
            elif  ord.at[sy,'Order']==0 and (vhod1m==-1 and vhod5m==-1 and vhod15m!=1) and vhod1h!=1   and vhod4h!=1 :
                open_position(symb,'short',2*abs(float(tks.at[sy,'minOrd'])))
                ord.at[sy,'Order']=-2*float(tks.at[sy,'minOrd'])
                ord.at[sy,'OrderTime']=str(time.ctime()) 
                print('                       '+symb+' Зашли SELL '+str(get_symbol_price(symb))+' Order= '+str(ord.at[sy,'Order'])+'  Signal= '+ str(ord.at[sy,'Signal'])+str(time.ctime()))
                print(' '+str(vhod1m)+'  '+str(vhod5m)+'  '+str(vhod15m)+'  '+str(vhod1h)+'  '+str(vhod4h))
              #sendImage(symb+timefr)


            elif ord.at[sy,'Order']>0 and (vhod1m<0 and vhod5m<0) :# and vhod15m<1  and ( vhod1h<1 or vhod4h<1  ) :
                open_position(symb,'short',abs(float(ord.at[sy,'Order'])))
                ord.at[sy,'Order']=0
                ord.at[sy,'OrderTime']=str(time.ctime())                 
                print(symb+' Выходим из лонга  '+str(finrez)+str(get_symbol_price(symb))+'  Order= '+str(ord.at[sy,'Order'])+'  Signal= '+ str(ord.at[sy,'Signal'])+str(time.ctime()))
               
            elif ord.at[sy,'Order']<0 and (vhod1m>0 and vhod5m>0) :# and vhod15m>-1 and ( vhod1h>-1 or vhod4h>-1  ):
                open_position(symb,'long',abs(float(ord.at[sy,'Order'])))
                ord.at[sy,'Order']=0
                ord.at[sy,'OrderTime']=str(time.ctime())
                print(symb+' Выходим из шорта   '+str(finrez)+str(get_symbol_price(symb))+'  Order= '+str(ord.at[sy,'Order'])+'  Signal= '+ str(ord.at[sy,'Signal'])+str(time.ctime()))
              
        #tks.at[sy,'Sum']=sum(tks.loc[sy][1:-1]) 
        tks.to_csv(path+'SignalsymbFuture.csv')
        #ord.to_csv(path+'OrdersFutures.csv')
          

Text(value='', disabled=True)

Textarea(value='', disabled=True)

Textarea(value='', disabled=True)

Textarea(value='', disabled=True)

ToggleButton(value=True, description='Stop', disabled=True, icon='check', tooltip='Description')

...

C:\Temp/ipykernel_1664/1532189744.py:707: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access

C:\Temp/ipykernel_1664/1532189744.py:707: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access

C:\Temp/ipykernel_1664/1532189744.py:707: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access

C:\Temp/ipykernel_1664/1532189744.py:707: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access

C:\Temp/ipykernel_1664/1532189744.py:707: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stab

Sat Apr  2 12:52:28 2022 ATOMUSDT 1 -1 0 1 0 30.36


C:\Temp/ipykernel_1664/1532189744.py:707: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access

C:\Temp/ipykernel_1664/1532189744.py:707: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access

C:\Temp/ipykernel_1664/1532189744.py:707: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access

C:\Temp/ipykernel_1664/1532189744.py:707: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access

C:\Temp/ipykernel_1664/1532189744.py:707: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stab

Исключение


KeyboardInterrupt: 

In [ ]:
long

In [ ]:
rol1=pd.DataFrame()   
tr=['1m','5m','15m','1h','4h']
for tfr in tr:
    df2=get_futures_klines(symb,tfr,500)
    for k in range(1,8):
        rol1.at[k,tfr]=bactest(df2,k,1)            
rol1
max(rol1)

In [ ]:
rol1[rol1.sum().idxmax()].idxmax()

# bactest History


In [ ]:
### HIstory Back test

def SignalTest(df):
    ohlc=df
    sb = ohlc.long2
    ss = ohlc.short2
    

    long= (sb==1)&(sb.shift()==0)
    short =  (ss==1)&(ss.shift()==0)
    if long[-2]==1  or  long[-1]==1:
        return 1
    elif short[-2]==1 or short[-1]==1  :
        return -1 
    else:
        return 0
### HIstory Back test
def SignalTest1(df,sy,i):
    s=0  
    if df.DeltCO[(df.index[i])]>0  :#\
            Testord.at[sy,'SignalTime']=df.open_time[(df.index[i])]  
            Testord.at[sy,'Signal']=1
            #print(symb+str(tks.at[sy,'deltaVol'])+ ' = DeltaVol   Signal= '+ str(ord.at[sy,'Signal'])+' Order= '+str(ord.at[sy,'Order']))
            #print(' ***покупаем на все*** '+siga)
            s=1
            return s

    elif  df.DeltCO[(df.index[i])]<0 :
            Testord.at[sy,'Signal']=-1
            Testord.at[sy,'SignalTime']=df.open_time[(df.index[i])]  
            #print('                               ***продаем на все*** '+siga)
            s=-1
            return s
    else:# df.DeltHC[(df.index[i])]>df.DeltCL[(df.index[i])] and df.DeltHO[(df.index[i])]>df.DeltOL[(df.index[i])] \
    #or df.DeltHC[(df.index[i])]<df.DeltCL[(df.index[i])] and df.DeltHO[(df.index[i])]<df.DeltOL[(df.index[i])]:
            Testord.at[sy,'Signal']=0
            Testord.at[sy,'SignalTime']=df.open_time[(df.index[i])]        
            #print('                                                          Шляпа какая-то '+siga)
            return s
    Testord.at[sy,'Signal']=0
    return s



Testord[['Signal', 'SignalPrice','Order','OrderPrice','FinRez','MaxFinRez']] = 0.0


t=0 
if t==0:#def backtest()
    
    rezult=0.0
    timefr='5m'
for sy in Testord.loc[:].index:
    
    symb=Testord.symbol[sy]
    df77=get_futures_klines(symb,timefr,1500)
    
    df777=pd.concat([df77, pd.DataFrame(columns = ['vhod4h', 'vhod1h','vhod15m','vhod5m','vhod1m',
                                                   'prbuy','prsell','comiss','profit','otkrb', 'orkrs', 
                                                   'zakr','perev','prib'])])
    print(tks.symbol[sy]+str(time.ctime()))
    profit=0.0
    prib=0
    zakr=0
    otkrb=0
    otkrs=0
    perev=0
    pribR=0
    pribL=0
    pribT=0
    i=-1 # по последней свече анализируем в индексе
    #@jit(nopython=True)
    #def testik(symb,df77,df777):
        
    for n in (df77.index):
         if n>df77[300:301].index:
            df=df77.loc[:n]
            
            df4h=tfizm5(df,120)
            df4h=podgData(df4h,5)
            vhod4h=Signal(df4h)
 
            df1h=tfizm5(df,60)
            df1h=podgData(df1h)
            #if bactest(df1h,1)>0:
            vhod1h=Signal(df1h)
            
            df1m=podgData(df.tail(20))
            vhod1m=Signal(df1m)
            
            df5m=tfizm5(df,4)
            df5m=podgData(df5m.tail(20))
            #if bactest(df5m,1)>0:
            vhod5m=Signal(df5m) 
            
            
            df15m=tfizm5(df,15)
            df15m=podgData(df15m.tail(20))
            #if bactest(df5m,1)>0:
            vhod15m=Signal(df15m) 
            
            prbuy= round(float(df.C[(df.index[i])])*abs(float(Testord.at[sy,'Order']))-(float(Testord.at[sy,'OrderPrice'])*abs(float(Testord.at[sy,'Order']))),2)
            prsell= round(float(Testord.at[sy,'OrderPrice'])*abs(float(Testord.at[sy,'Order']))-(float(df.C[(df.index[i])])*abs(float(Testord.at[sy,'Order']))),2)
            prbuy1= round(float(df.C[(df.index[i-1])])*abs(float(Testord.at[sy,'Order']))-(float(Testord.at[sy,'OrderPrice'])*abs(float(Testord.at[sy,'Order']))),2)
            prsell1= round(float(Testord.at[sy,'OrderPrice'])*abs(float(Testord.at[sy,'Order']))-(float(df.C[(df.index[i-1])])*abs(float(Testord.at[sy,'Order']))),2)
            
            comiss=round(0.001*float(df.C[(df.index[i])])*abs(float(Testord.at[sy,'Order'])),2)
            
            #print(prbuy,prsell,comiss)
            if Testord.at[sy,'Order']==0 and vhod1m==1 and vhod5m==1:# and vhod15m==1 and vhod1h!=-1 and vhod4h!=-1 :
                Testord.at[sy,'Order']=2*float(Testord.at[sy,'minOrd'])
                Testord.at[sy,'OrderTime']=df.close_time[(df.index[i])]
                Testord.at[sy,'OrderPrice']=   df.C[(df.index[i])]
                #print(round(profit,2), Testord.at[sy,'OrderPrice'],Testord.at[sy,'OrderTime'])
                #print('зашли в лонг')
                otkrb+=1
            elif Testord.at[sy,'Order']==0 and vhod1m==-1 and vhod5m==-1:# and vhod15m==-1 and vhod1h!=1 and vhod4h!=1 :
                Testord.at[sy,'Order']=-2*float(Testord.at[sy,'minOrd'])
                Testord.at[sy,'OrderTime']=df.close_time[(df.index[i])]
                Testord.at[sy,'OrderPrice']=df.C[(df.index[i])]
                #print(round(profit,2), Testord.at[sy,'OrderPrice'],Testord.at[sy,'OrderTime'])
                #print('зашли в шорт')
                otkrs+=1

            elif  Testord.at[sy,'Order']>0 and  ((0<prbuy<prbuy1) or (vhod1m<0 and vhod5m<0)): 
                profit=profit+prbuy-comiss
                
                Testord.at[sy,'Order']=-2*float(Testord.at[sy,'minOrd'])
                Testord.at[sy,'OrderTime']=df.close_time[(df.index[i])]
                Testord.at[sy,'OrderPrice']=df.C[(df.index[i])]  
                #print(round(profit,2),Testord.at[sy,'OrderTime'],prbuy)
                zakr+=1            
            elif  Testord.at[sy,'Order']<0 and ( (0<prsell<prsell1) or (vhod1m>0 and vhod5m>0)):# and vhod15m>-1 and (vhod1h>-1 or vhod4h>-1):
                profit=profit+prsell-comiss
                
                Testord.at[sy,'Order']=2*float(Testord.at[sy,'minOrd'])
                Testord.at[sy,'OrderTime']=df.close_time[(df.index[i])]
                Testord.at[sy,'OrderPrice']=df.C[(df.index[i])]
                #print(round(profit,2),Testord.at[sy,'OrderTime'],prsell)
                zakr+=1 

            ### Reducir profit
            if Testord.at[sy,'Order']>0 and Testord.at[sy,'Order']==2*float(Testord.at[sy,'minOrd'])  and \
            (round(float(df.C[(df.index[i])])*abs(float(Testord.at[sy,'Order']))-(
                float(Testord.at[sy,'OrderPrice'])*abs(float(Testord.at[sy,'Order']))),2)>0.2 or\
             round(float(df.C[(df.index[i])])*abs(float(Testord.at[sy,'Order']))-(
                float(Testord.at[sy,'OrderPrice'])*abs(float(Testord.at[sy,'Order']))),2)<-0.07) :
                profit=profit+round(float(df.C[(df.index[i])])*abs(float(Testord.at[sy,'Order']))-(float(Testord.at[sy,'OrderPrice'])*abs(float(Testord.at[sy,'Order']))),2)*0.5-round(0.001*float(df.C[(df.index[i])])*abs(float(Testord.at[sy,'Order'])),2)*0.5
                Testord.at[sy,'Order']=Testord.at[sy,'Order']*0.5
                Testord.at[sy,'OrderTime']=df.close_time[(df.index[i])]
                #print(round(profit,2),Testord.at[sy,'OrderTime'],prbuy)
                #print('reducir profit long')
                pribR+=1 
            if Testord.at[sy,'Order']<0 and Testord.at[sy,'Order']==-2*float(Testord.at[sy,'minOrd']) and \
            (round(float(Testord.at[sy,'OrderPrice'])*abs(float(Testord.at[sy,'Order']))-(
                float(df.C[(df.index[i])])*abs(float(Testord.at[sy,'Order']))),2)>0.2 or\
             round(float(Testord.at[sy,'OrderPrice'])*abs(float(Testord.at[sy,'Order']))-(
                float(df.C[(df.index[i])])*abs(float(Testord.at[sy,'Order']))),2)<-0.7)  :
                profit=profit+round(float(Testord.at[sy,'OrderPrice'])*abs(float(Testord.at[sy,'Order']))-(float(df.C[(df.index[i])])*abs(float(Testord.at[sy,'Order']))),2)*0.5-round(0.001*float(df.C[(df.index[i])])*abs(float(Testord.at[sy,'Order'])),2)*0.5
                Testord.at[sy,'Order']=Testord.at[sy,'Order']*0.5
                Testord.at[sy,'OrderTime']=df.close_time[(df.index[i])]
                #print(round(profit,2),Testord.at[sy,'OrderTime'],prsell)
                #print('reducir profit short')
                pribR+=1 

            ### TAKE PROFIT
            if Testord.at[sy,'Order']>0 and \
            (round(float(df.C[(df.index[i])])*abs(float(Testord.at[sy,'Order']))-(
                float(Testord.at[sy,'OrderPrice'])*abs(float(Testord.at[sy,'Order']))),2)>0.6) :
                profit=profit+round(float(df.C[(df.index[i])])*abs(float(
                    Testord.at[sy,'Order']))-(float(Testord.at[sy,'OrderPrice'])*abs(
                    float(Testord.at[sy,'Order']))),2)-round(0.001*float(df.C[(df.index[i])])*abs(
                    float(Testord.at[sy,'Order'])),2)
                Testord.at[sy,'Order']=0
                Testord.at[sy,'OrderTime']=df.close_time[(df.index[i])]
                Testord.at[sy,'OrderPrice']=0
                #print(round(profit,2),Testord.at[sy,'OrderTime'],prbuy)
                #print('TAKE PROFIT')
                pribT+=1 
            if Testord.at[sy,'Order']<0 and \
            (round(float(Testord.at[sy,'OrderPrice'])*abs(float(Testord.at[sy,'Order']))-(float(df.C[(df.index[i])])*abs(float(Testord.at[sy,'Order']))),2)>0.6)  :
                profit=profit+round(float(Testord.at[sy,'OrderPrice'])*abs(float(Testord.at[sy,'Order']))-(float(df.C[(df.index[i])])*abs(float(Testord.at[sy,'Order']))),2)-round(0.001*float(df.C[(df.index[i])])*abs(float(Testord.at[sy,'Order'])),2)
                Testord.at[sy,'Order']=0
                Testord.at[sy,'OrderTime']=df.close_time[(df.index[i])]
                Testord.at[sy,'OrderPrice']=0
                #print(round(profit,2),Testord.at[sy,'OrderTime'],prsell)
                #print('TAKE PROFIT')
                pribT+=1 

            ### STOP LOSS
            if Testord.at[sy,'Order']>0 and \
            (round(float(df.C[(df.index[i])])*abs(float(Testord.at[sy,'Order']))-(float(Testord.at[sy,'OrderPrice'])*abs(float(Testord.at[sy,'Order']))),2)<-0.2) :
                profit=profit+round(float(df.C[(df.index[i])])*abs(float(Testord.at[sy,'Order']))-(float(Testord.at[sy,'OrderPrice'])*abs(float(Testord.at[sy,'Order']))),2)-round(0.001*float(df.C[(df.index[i])])*abs(float(Testord.at[sy,'Order'])),2)
                Testord.at[sy,'Order']=0
                Testord.at[sy,'OrderTime']=df.close_time[(df.index[i])]
                Testord.at[sy,'OrderPrice']=0
                #print(round(profit,2),Testord.at[sy,'OrderTime'],prbuy)
                #print('STOP LOSS')
                pribL+=1 
            if Testord.at[sy,'Order']<0 and\
             (round(float(Testord.at[sy,'OrderPrice'])*abs(float(Testord.at[sy,'Order']))-(float(df.C[(df.index[i])])*abs(float(Testord.at[sy,'Order']))),2)<-0.2)  :
                profit=profit+round(float(Testord.at[sy,'OrderPrice'])*abs(float(Testord.at[sy,'Order']))-(float(df.C[(df.index[i])])*abs(float(Testord.at[sy,'Order']))),2)-round(0.001*float(df.C[(df.index[i])])*abs(float(Testord.at[sy,'Order'])),2)
                Testord.at[sy,'Order']=0
                Testord.at[sy,'OrderTime']=df.close_time[(df.index[i])]
                Testord.at[sy,'OrderPrice']=0
                #print(round(profit,2),Testord.at[sy,'OrderTime'],prsell)
                #print('STOP LOSS')
                pribL+=1                

            df777.at[n,'vhod1m']=vhod1m
            df777.at[n,'vhod5m']=vhod5m
            df777.at[n,'vhod15m']=vhod15m
            df777.at[n,'vhod1h']=vhod1h
            df777.at[n,'vhod4h']=vhod4h
            df777.at[n,'otkrb']=otkrb
            df777.at[n,'otkrs']=otkrs
            df777.at[n,'zakr']=zakr
            df777.at[n,'profit']=profit
            df777.at[n,'prib']=prib
            df777.at[n,'perev']=perev
            print(vhod1m,vhod5m,vhod15m,vhod1h,vhod4h,df.C[(df.index[i])],df.close_time[(df.index[i])],round(profit,2), otkrb,otkrs,zakr,pribT,pribR,pribL,perev,prbuy,prsell)
            

   
            #df[df.index==n] [ ['sig4h', 'sig1h','sigvh','s','vhod15m','vhod1h','prbuy','prsell','comiss','profit']]=[[sig4h, sig1h,sigvh,s,vhod15m,vhod1h,prbuy,prsell,comiss,profit]]  
            #df777.to_csv(path+'BackTest'+symb+timefr+'.csv')

    Testord.at[sy,'FinRez']=profit
    print(round(profit,2),otkrb,zakr,perev,prib,n)
     
    rezult=rezult+profit
    print(rezult)

In [ ]:
df77[3:4].index

In [ ]:
df77

# backtest online

In [ ]:
# ONLINE TEST
def SignalTest(df,sy,i):
    s=0  
    if df.DeltCL[(df.index[i-1])]<df.DeltCL[(df.index[i])] and df.DeltOL[(df.index[i])]<df.DeltCL[(df.index[i])]  :#\
            Testord.at[sy,'SignalTime']=df.open_time[(df.index[i])]  
            Testord.at[sy,'Signal']=1
            #print(symb+str(tks.at[sy,'deltaVol'])+ ' = DeltaVol   Signal= '+ str(ord.at[sy,'Signal'])+' Order= '+str(ord.at[sy,'Order']))
            #print(' ***покупаем на все*** '+siga)
            s=1
            return s

    elif  df.DeltHC[(df.index[i])]>df.DeltHO[(df.index[i])] and df.DeltHC[(df.index[i])]>df.DeltHC[(df.index[i-1])] :
            Testord.at[sy,'Signal']=-1
            Testord.at[sy,'SignalTime']=df.open_time[(df.index[i])]  
            #print('                               ***продаем на все*** '+siga)
            s=-1
            return s
    else:# df.DeltHC[(df.index[i])]>df.DeltCL[(df.index[i])] and df.DeltHO[(df.index[i])]>df.DeltOL[(df.index[i])] \
    #or df.DeltHC[(df.index[i])]<df.DeltCL[(df.index[i])] and df.DeltHO[(df.index[i])]<df.DeltOL[(df.index[i])]:
            Testord.at[sy,'Signal']=0
            Testord.at[sy,'SignalTime']=df.open_time[(df.index[i])]        
            #print('                                                          Шляпа какая-то '+siga)
            return s
    Testord.at[sy,'Signal']=0
    return s



Testord[['Signal', 'SignalPrice','Order','OrderPrice','FinRez','MaxFinRez']] = 0.0
rezult=0.0
timefr='5m'

prib=0
zakr=0
otkr=0
perev=0
pribR=0
pribL=0
pribT=0
  #df=pd.read_csv(Hist+symb+timefr+'Obrabotka.csv')
profit=0.0
 
while True:
  

  nn=[0.1,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.60,0.7,0.8,0.9,1,1.5,2,2.5,3]
  n=0.2
  for sy in tks.loc[:].index:
    symb=tks.symbol[sy]
    df77=podgData(get_futures_klines(symb,timefr,350))
    df777=pd.concat([df77, pd.DataFrame(columns = ['sig4h', 'sig1h','sigvh','vhod5m','vhod15m','vhod1h','vhod4h','prsell','comiss','profit','otkr','zakr','perev','prib'])])
    prise=0
    print("    "+tks.symbol[sy])
    i=-1
    df5=df77 #tfizm5(df77,5)
    df4h=podgData(tfizm5(df5,48))
    vhod4h=SignalTest(df4h,sy,i)
    sig4h=SignalVh(df4h)
    
    df1h=podgData(tfizm5(df5,12))
    vhod1h=SignalTest(df1h,sy,i)
    sig1h=SignalVh(df1h)

    df15m=podgData(tfizm5(df5,3))
    vhod15m=SignalTest(df15m,sy,i)

    prise=float(df.close[(df.index[i])])#get_symbol_price(symb)
    #SignalVh(df1h)
    df=podgData(df5)
    #Signal(df,sy,i)
    sigvh=SignalVh(df)
    #print(vhod4h,vhod1h,vhod15m,vhod5m)    
    vhod5m=SignalTest(df,sy,i)

    prbuy= round(float(df.close[(df.index[i])])*abs(float(Testord.at[sy,'Order']))-(float(Testord.at[sy,'OrderPrice'])*abs(float(Testord.at[sy,'Order']))),2)
    prsell= round(float(Testord.at[sy,'OrderPrice'])*abs(float(Testord.at[sy,'Order']))-(float(df.close[(df.index[i])])*abs(float(Testord.at[sy,'Order']))),2)
    comiss=round(0.001*float(df.close[(df.index[i])])*abs(float(Testord.at[sy,'Order'])),2)

    if i==-1:
            if Testord.at[sy,'Signal']>0 and Testord.at[sy,'Order']==0 and sigvh==1  and vhod15m!=-1 and vhod1h!=-1:# and vhod4h!=-1:# and vhod15m>0 and vhod1h>0and vhod4h!=0 and sigvh==1  :
                Testord.at[sy,'Order']=2*float(Testord.at[sy,'minOrd'])
                Testord.at[sy,'OrderTime']=df.open_time[(df.index[i])]
                Testord.at[sy,'OrderPrice']=   prise
                #print(round(profit,2), Testord.at[sy,'OrderPrice'],Testord.at[sy,'OrderTime'])
                #print('зашли в лонг')
                otkr+=1
            elif Testord.at[sy,'Signal']<0 and Testord.at[sy,'Order']==0 and sigvh==1 and vhod15m!=1 and vhod1h!=1:# and vhod4h!=1:# and vhod15m>0 and vhod1h>0and vhod4h!=0 and sigvh==1:
                Testord.at[sy,'Order']=-2*float(Testord.at[sy,'minOrd'])
                Testord.at[sy,'OrderTime']=df.open_time[(df.index[i])]
                Testord.at[sy,'OrderPrice']=prise
                #print(round(profit,2), Testord.at[sy,'OrderPrice'],Testord.at[sy,'OrderTime'])
                #print('зашли в шорт')
                otkr+=1

            elif  Testord.at[sy,'Order']>0  :
                print(Testord.at[sy,'Order'],round((prise*abs(Testord.at[sy,'Order']))-(Testord.at[sy,'OrderPrice']*abs(Testord.at[sy,'Order'])),2)-round(0.001*prise*abs(float(Testord.at[sy,'Order'])),2))                
            elif  Testord.at[sy,'Order']<0  :
                print(Testord.at[sy,'Order'],round(-(prise*abs(Testord.at[sy,'Order']))+(Testord.at[sy,'OrderPrice']*abs(Testord.at[sy,'Order'])),2) -round(0.001*prise*abs(float(Testord.at[sy,'Order'])),2))                


            elif (Testord.at[sy,'Signal']==0 and Testord.at[sy,'Order']>0 and (vhod15m==-1 or vhod1h==-1 )) :# and (vhod15m==0 or vhod1h==0):
                profit=profit+round((prise*abs(Testord.at[sy,'Order']))-(Testord.at[sy,'OrderPrice']*abs(Testord.at[sy,'Order'])),2)-round(0.001*prise*abs(float(Testord.at[sy,'Order'])),2) 
                Testord.at[sy,'Order']=0
                Testord.at[sy,'OrderTime']=df.open_time[(df.index[i])]
                Testord.at[sy,'OrderPrice']=0  
                #print(round(profit,2),Testord.at[sy,'OrderTime'],prbuy)
                zakr+=1            
            elif Testord.at[sy,'Signal']==0 and Testord.at[sy,'Order']<0 and (vhod15m==1 or vhod1h==1) :#and (vhod15m==0 or vhod1h==0): 
                profit=profit+round(-(prise*abs(Testord.at[sy,'Order']))+(Testord.at[sy,'OrderPrice']*abs(Testord.at[sy,'Order'])),2) -round(0.001*prise*abs(float(Testord.at[sy,'Order'])),2)
                Testord.at[sy,'Order']=0
                Testord.at[sy,'OrderTime']=df.open_time[(df.index[i])]
                Testord.at[sy,'OrderPrice']=0
                #print(round(profit,2),Testord.at[sy,'OrderTime'],prsell)
                zakr+=1 

            elif Testord.at[sy,'Signal']<0 and Testord.at[sy,'Order']>0  and (vhod15m!=1 and vhod1h!=1):
                profit=profit+round((prise*abs(Testord.at[sy,'Order']))-(Testord.at[sy,'OrderPrice']*abs(Testord.at[sy,'Order'])),2)-round(0.001*prise*abs(float(Testord.at[sy,'Order'])),2) 
                Testord.at[sy,'Order']=0#-float(Testord.at[sy,'Order'])
                Testord.at[sy,'OrderTime']=df.open_time[(df.index[i])]
                Testord.at[sy,'OrderPrice']=0#df.close[(df.index[i])]
                #print(round(profit,2),Testord.at[sy,'OrderTime'],Testord.at[sy,'Order'],prbuy)
                #print('закрыли лонг,сигнал в шорт')
                perev+=1
            elif Testord.at[sy,'Signal']>0 and Testord.at[sy,'Order']<0  and (vhod15m!=-1 and vhod1h!=-1):
                profit=profit+round(-(prise*abs(Testord.at[sy,'Order']))+(Testord.at[sy,'OrderPrice']*abs(Testord.at[sy,'Order'])),2) -round(0.001*prise*abs(float(Testord.at[sy,'Order'])),2)
                Testord.at[sy,'Order']=0#abs(float(Testord.at[sy,'Order']))
                Testord.at[sy,'OrderTime']=df.open_time[(df.index[i])]
                Testord.at[sy,'OrderPrice']=0#df.close[(df.index[i])]
                #print(round(profit,2),Testord.at[sy,'OrderTime'],Testord.at[sy,'Order'],prsell)
                #print('закрыли шорт сигнал в лонг')
                perev+=1   

                  ### Reducir profit
            elif Testord.at[sy,'Order']>0 and Testord.at[sy,'Order']==2*float(Testord.at[sy,'minOrd'])  and \
               (round((prise*abs(Testord.at[sy,'Order']))-(Testord.at[sy,'OrderPrice']*abs(Testord.at[sy,'Order'])),2)>0.3 \
                or round((prise*abs(Testord.at[sy,'Order']))-(Testord.at[sy,'OrderPrice']*abs(Testord.at[sy,'Order'])),2)<-0.05) :
                profit=profit+round((prise*abs(Testord.at[sy,'Order']))-(Testord.at[sy,'OrderPrice']*abs(Testord.at[sy,'Order'])),2)*0.5-round(0.001*prise*abs(float(Testord.at[sy,'Order'])),2)*0.5 
                Testord.at[sy,'Order']=Testord.at[sy,'Order']*0.5
                Testord.at[sy,'OrderTime']=df.open_time[(df.index[i])]
                #print(round(profit,2),Testord.at[sy,'OrderTime'],prbuy)
                #print('reducir profit long')
                pribR+=1 
            elif Testord.at[sy,'Order']<0 and Testord.at[sy,'Order']==-2*float(Testord.at[sy,'minOrd']) and (round(-(prise*abs(Testord.at[sy,'Order']))+(Testord.at[sy,'OrderPrice']*abs(Testord.at[sy,'Order'])),2)>0.3 or round(-(prise*abs(Testord.at[sy,'Order']))+(Testord.at[sy,'OrderPrice']*abs(Testord.at[sy,'Order'])),2)<-0.05)  :
                profit=profit+round(-(prise*abs(Testord.at[sy,'Order']))+(Testord.at[sy,'OrderPrice']*abs(Testord.at[sy,'Order'])),2)*0.5 -round(0.001*prise*abs(float(Testord.at[sy,'Order'])),2)*0.5
                Testord.at[sy,'Order']=Testord.at[sy,'Order']*0.5
                Testord.at[sy,'OrderTime']=df.open_time[(df.index[i])]
                #print(round(profit,2),Testord.at[sy,'OrderTime'],prsell)
                #print('reducir profit short')
                pribR+=1 

                 ### TAKE PROFIT
            elif Testord.at[sy,'Order']>0 and (round((prise*abs(Testord.at[sy,'Order']))-(Testord.at[sy,'OrderPrice']*abs(Testord.at[sy,'Order'])),2)>1.3) :
                profit=profit+round((prise*abs(Testord.at[sy,'Order']))-(Testord.at[sy,'OrderPrice']*abs(Testord.at[sy,'Order'])),2)-round(0.001*prise*abs(float(Testord.at[sy,'Order'])),2) 
                Testord.at[sy,'Order']=0
                Testord.at[sy,'OrderTime']=df.open_time[(df.index[i])]
                Testord.at[sy,'OrderPrice']=0
                #print(round(profit,2),Testord.at[sy,'OrderTime'],prbuy)
                #print('TAKE PROFIT')
                pribT+=1 
            elif Testord.at[sy,'Order']<0 and (round(-(prise*abs(Testord.at[sy,'Order']))+(Testord.at[sy,'OrderPrice']*abs(Testord.at[sy,'Order'])),2)>1.3)  :
                profit=profit+round(-(prise*abs(Testord.at[sy,'Order']))+(Testord.at[sy,'OrderPrice']*abs(Testord.at[sy,'Order'])),2) -round(0.001*prise*abs(float(Testord.at[sy,'Order'])),2)
                Testord.at[sy,'Order']=0
                Testord.at[sy,'OrderTime']=df.open_time[(df.index[i])]
                Testord.at[sy,'OrderPrice']=0
                #print(round(profit,2),Testord.at[sy,'OrderTime'],prsell)
                #print('TAKE PROFIT')
                pribT+=1 

                 ### STOP LOSS
            elif Testord.at[sy,'Order']>0 and (round((prise*abs(Testord.at[sy,'Order']))-(Testord.at[sy,'OrderPrice']*abs(Testord.at[sy,'Order'])),2)<-0.5) :
                profit=profit+round((prise*abs(Testord.at[sy,'Order']))-(Testord.at[sy,'OrderPrice']*abs(Testord.at[sy,'Order'])),2)-round(0.001*prise*abs(float(Testord.at[sy,'Order'])),2) 
                Testord.at[sy,'Order']=0
                Testord.at[sy,'OrderTime']=df.open_time[(df.index[i])]
                Testord.at[sy,'OrderPrice']=0
                #print(round(profit,2),Testord.at[sy,'OrderTime'],prbuy)
                #print('STOP LOSS')
                pribL+=1 
            elif Testord.at[sy,'Order']<0 and (round(-(prise*abs(Testord.at[sy,'Order']))+(Testord.at[sy,'OrderPrice']*abs(Testord.at[sy,'Order'])),2)<-0.5)  :
                profit=profit+round(-(prise*abs(Testord.at[sy,'Order']))+(Testord.at[sy,'OrderPrice']*abs(Testord.at[sy,'Order'])),2) -round(0.001*prise*abs(float(Testord.at[sy,'Order'])),2)
                Testord.at[sy,'Order']=0
                Testord.at[sy,'OrderTime']=df.open_time[(df.index[i])]
                Testord.at[sy,'OrderPrice']=0
                #print(round(profit,2),Testord.at[sy,'OrderTime'],prsell)
                #print('STOP LOSS')
                pribL+=1                
            df777.at[n,'sig4h']=sig4h
            df777.at[n,'sig1h']=sig1h
            df777.at[n,'sigvh']=sig1h
            df777.at[n,'vhod5m']=vhod5m
            df777.at[n,'vhod15m']=vhod15m
            df777.at[n,'vhod1h']=vhod1h
            df777.at[n,'vhod4h']=vhod4h
            df777.at[n,'otkr']=otkr
            df777.at[n,'zakr']=zakr
            df777.at[n,'profit']=profit
            df777.at[n,'prib']=prib
            df777.at[n,'perev']=perev
            print(sigvh,vhod5m,vhod15m,vhod1h,vhod4h,prise,str(time.ctime()),round(profit,2), otkr,zakr,pribT,pribR,pribL,perev,profit)
            

            Testord.at[sy,'FinRez']=profit
            #df[df.index==n] [ ['sig4h', 'sig1h','sigvh','s','vhod15m','vhod1h','prbuy','prsell','comiss','profit']]=[[sig4h, sig1h,sigvh,s,vhod15m,vhod1h,prbuy,prsell,comiss,profit]]  
            #df777.to_csv(path+'BackTest'+symb+timefr+'2.csv')
            rezult=rezult+profit
            print(rezult,profit)
 

In [ ]:
Testord.at[sy,'Order']

In [ ]:
symb

In [ ]:
prsell

In [ ]:
round(-(prise*abs(Testord.at[sy,'Order']))+(Testord.at[sy,'OrderPrice']*abs(Testord.at[sy,'Order'])),2) -round(0.001*prise*abs(float(Testord.at[sy,'Order'])),2)

In [ ]:
prise

In [ ]:
Testord.at[sy,'OrderPrice']

In [ ]:
#@title
var = 2 #@param {type:"integer"}
#  ISHODNOE
#                         ОСНОВНОЕ ПРОВЕРКА И ТОРГОВЛЯ
#
while True:
    for sy in tks.loc[:30].index:
        if sy%7==0:
          reducir()
          
        
        status = client.futures_account()
        positions=pd.DataFrame(status['positions'])     
        vhod1h=0
        vhod15m=0
        vhod4h=0     
        symb=tks.symbol[sy]
        print('              '+ symb+'                        '+str(time.ctime()))
        order = positions[positions['symbol']==symb]['positionAmt'].astype(float).tolist()[0]
        finrez=positions[positions['symbol']==symb]['unrealizedProfit'].astype(float).tolist()[0]
        notional = abs(positions[positions['symbol']==symb]['notional'].astype(float).tolist()[0])
        ord.at[sy,'OrderPrice']  = positions[positions['symbol']==symb]['entryPrice']
        profit = float(status['totalUnrealizedProfit'])
        balance = round(float(status['totalWalletBalance']),2)
        ord.at[sy,'OrderPrice']  = positions[positions['symbol']==symb]['entryPrice']
        ord.at[sy,'Order'] = order
        ord.at[sy,'FinRez'] = finrez   
        print(' profit= '+str(profit)+'  balance= '+str(balance))     
        timefr='5m'        
        #analiz
        if  timefr=='5m':
            
            df1=get_futures_klines(symb,timefr,1500)
            df5=tfizm5(df1,5)

            df4h=podgData(tfizm5(df5,48))
            vhod4h=Signal(df4h,sy,-1)
            #SignalVh(df4h)
            
            df1h=podgData(tfizm5(df5,12))
            vhod1h=Signal(df1h,sy,-1)
            #SignalVh(df1h)

            df15m=podgData(tfizm5(df5,3))
            vhod15m=Signal(df15m,sy,-1)
            #SignalVh(df1h)
            df1= podgData(df1)
            vhod= Signal(df1,sy,-1)
            sigvh1=0#SignalVh(df1)

            df=podgData(df5)
            zapisHistory(df,timefr,symb)
            vhod5m=Signal(df,sy,-1)
            sigvh=0#SignalVh(df)
            


            print(sigvh,sigvh1,vhod,vhod5m,vhod15m,vhod1h,vhod4h)            
            i=-1
            
            if ord.at[sy,'Signal']>0 and ord.at[sy,'Order']==0  and vhod15m!=-1 and vhod1h!=-1:
              open_position(symb,'long',2*abs(float(tks.at[sy,'minOrd'])))
              ord.at[sy,'Order']=2*float(tks.at[sy,'minOrd'])
              ord.at[sy,'OrderTime']=str(time.ctime())
              print(symb+' Зашли BUY '+str(get_symbol_price(symb))+' Order= '+str(ord.at[sy,'Order'])+'  Signal= '+ str(ord.at[sy,'Signal'])+str(time.ctime()))
              chartTel(df.tail(100),symb+timefr,'  Зашли BUY ')
              sendImage(symb+timefr)
                              
            elif ord.at[sy,'Signal']<0 and ord.at[sy,'Order']==0 and sigvh==1 and vhod15m!=1 and vhod1h!=1:
              open_position(symb,'short',2*abs(float(tks.at[sy,'minOrd'])))
              ord.at[sy,'Order']=-2*float(tks.at[sy,'minOrd'])
              ord.at[sy,'OrderTime']=str(time.ctime()) 
              print('                                                  '+symb+' Зашли SELL '+str(get_symbol_price(symb))+' Order= '+str(ord.at[sy,'Order'])+'  Signal= '+ str(ord.at[sy,'Signal'])+str(time.ctime()))
              chartTel(df.tail(100),symb+timefr,'  Зашли SELL ')
              sendImage(symb+timefr)


            elif ord.at[sy,'Signal']==0 and ord.at[sy,'Order']>0 and (vhod15m==-1 or vhod1h==-1 ) :
              open_position(symb,'short',abs(float(ord.at[sy,'Order'])))
              ord.at[sy,'Order']=0
              ord.at[sy,'OrderTime']=str(time.ctime())                 
              print(symb+' Выходим из лонга на М5 флэт '+str(get_symbol_price(symb))+'  Order= '+str(ord.at[sy,'Order'])+'  Signal= '+ str(ord.at[sy,'Signal'])+str(time.ctime()))
               
            elif ord.at[sy,'Signal']==0 and ord.at[sy,'Order']<0 and (vhod15m==1 or vhod1h==1 ) :
              open_position(symb,'long',abs(float(ord.at[sy,'Order'])))
              ord.at[sy,'Order']=0
              ord.at[sy,'OrderTime']=str(time.ctime())
              print(symb+' Выходим из шорта на М5 флэт  '+str(get_symbol_price(symb))+'  Order= '+str(ord.at[sy,'Order'])+'  Signal= '+ str(ord.at[sy,'Signal'])+str(time.ctime()))


            elif ord.at[sy,'Signal']<0 and ord.at[sy,'Order']>0 and (vhod15m!=1 and vhod1h!=1) :
              open_position(symb,'short',abs(2*float(ord.at[sy,'Order'])))
              ord.at[sy,'Order']=-float(ord.at[sy,'Order'])
              ord.at[sy,'OrderTime']=str(time.ctime())  
              print('              '+symb+' Перевернулись в SELL '+str(get_symbol_price(symb))+'  Order= '+str(ord.at[sy,'Order'])+'  Signal= '+ str(ord.at[sy,'Signal']))                    
              print(str(time.ctime()))
              chartTel(df.tail(100),symb+timefr,'  Перевернулись в SELL ')
              sendImage(symb+timefr)
               
            elif ord.at[sy,'Signal']>0 and ord.at[sy,'Order']<0 and (vhod15m!=-1 and vhod1h!=-1):
              open_position(symb,'long',abs(2*float(ord.at[sy,'Order'])))
              ord.at[sy,'Order']=abs(float(ord.at[sy,'Order']))
              ord.at[sy,'OrderTime']=str(time.ctime()) 
              print(symb+' Перевернулись в BUY '+str(get_symbol_price(symb))+'  Order= '+str(ord.at[sy,'Order'])+'  Signal= '+ str(ord.at[sy,'Signal']))                   
              print(str(time.ctime()))
              chartTel(df.tail(100),symb+timefr,'  Перевернулись в БАЙ ')
              sendImage(symb+timefr)                    
                   
        #tks.at[sy,'Sum']=sum(tks.loc[sy][1:-1]) 
        tks.to_csv(path+'SignalsymbFuture.csv')
        ord.to_csv(path+'OrdersFutures.csv')
          